In [1]:
!pip install -U spacy
import spacy
!python -m spacy download en
sp = spacy.load("en_core_web_sm")
import csv
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import os
!pip install nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from IPython.display import display, HTML
import nltk
nltk.download('punkt')
nltk.download('stopwords')
!pip install -U gensim
import gensim
from scipy import spatial
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import time
from sklearn.decomposition import PCA
import re
from statistics import mean
import json
# to make our plot outputs appear and be stored within the notebook:
%matplotlib inline 
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 5)

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 40.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
output_df = pd.read_excel("output.xls")
text_df = pd.read_csv("18_093_attachment_text_Aug21.csv", encoding="ISO-8859-1")[["Date", "Attachment_text", "cn_doc_id", "Summary", "BRCID"]]

/tmp/ipykernel_1054565/2352958704.py:2: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,2

In [3]:
complete_data = pd.merge(output_df, text_df, left_on='CN_Doc_ID', right_on='cn_doc_id', how='right')
complete_data.sort_values("ethnicitycleaned", axis=0, inplace=True)

In [4]:
complete_data.rename(columns={"ethnicitycleaned": "Ethnicity", "Attachment_text": "Patient_Summary", "age": "Age", "Gender_ID": "Gender", "diagnosis_date": "Diagnosis_Date", "document_date": "Document_Date", "CN_Doc_ID": "unique_id"}, inplace=True)
complete_data = complete_data.dropna(subset=["Ethnicity", "Patient_Summary"])
# display(complete_data)
print(complete_data["Ethnicity"].value_counts())

Ethnicity
British (A)                    588
Any other ethnic group (S)     240
                              ... 
Chinese (R)                     14
White and Black African (E)     10
Name: count, Length: 16, dtype: int64


In [5]:
complete_data["Ethnicity"] = complete_data["Ethnicity"].replace("Any other black background (P)", "Black").replace("African (N)", "Black").replace("Caribbean (M)", "Black").replace("White and Black African (E)", "Mixed Black").replace("White and black Caribbean (D)", "Mixed Black")
complete_data["Ethnicity"] = complete_data["Ethnicity"].replace("Indian (H)", "South Asian").replace("Pakistani (J)", "Asian").replace("Bangladeshi (K)", "Asian")
complete_data["Ethnicity"] = complete_data["Ethnicity"].replace("Chinese (R)", "Asian").replace("Any other Asian background (L)", "Asian")
complete_data["Ethnicity"] = complete_data["Ethnicity"].replace("Any other white background (C)", "White").replace("Irish (B)", "White").replace("British (A)", "White")
complete_data["Ethnicity"] = complete_data["Ethnicity"].replace("Any other mixed background (G)", "Other")
print(complete_data["Ethnicity"].value_counts())

Ethnicity
White          743
Black          393
              ... 
Mixed Black     31
Other           19
Name: count, Length: 8, dtype: int64


In [6]:
complete_data["Patient_Summary"] = complete_data["Patient_Summary"].replace("dr ".casefold(), "doctor ")
complete_data["Patient_Summary"] = complete_data["Patient_Summary"].replace("bipolar disorder".casefold(), "bipolar_disorder")

In [7]:
white_data = complete_data[complete_data["Ethnicity"] == "White"]
black_data = complete_data[complete_data["Ethnicity"] == "Black"]
mixed_data = complete_data[complete_data["Ethnicity"] == "Mixed Black"]
asian_data = complete_data[complete_data["Ethnicity"] == "Asian"]

print(white_data.size)
print(black_data.size)
print(mixed_data.size)
print(asian_data.size)

# display(white_data)
# display(black_data)
# display(asian_data)

10402
5502
434
1470


In [23]:
white_data['Patient_Summary_sp'] = [sp(white_data['Patient_Summary'].iloc[i]) for i in range(white_data.shape[0])]
black_data['Patient_Summary_sp'] = [sp(black_data['Patient_Summary'].iloc[i]) for i in range(black_data.shape[0])]
mixed_data['Patient_Summary_sp'] = [sp(mixed_data['Patient_Summary'].iloc[i]) for i in range(mixed_data.shape[0])]
asian_data['Patient_Summary_sp'] = [sp(asian_data['Patient_Summary'].iloc[i]) for i in range(asian_data.shape[0])]
print("Finished running spacy")

white_data['tokens'] = [[word.text for word in white_data['Patient_Summary_sp'].iloc[i]] for i in range(white_data.shape[0])]
black_data['tokens'] = [[word.text for word in black_data['Patient_Summary_sp'].iloc[i]] for i in range(black_data.shape[0])]
mixed_data['tokens'] = [[word.text for word in mixed_data['Patient_Summary_sp'].iloc[i]] for i in range(mixed_data.shape[0])]
asian_data['tokens'] = [[word.text for word in asian_data['Patient_Summary_sp'].iloc[i]] for i in range(asian_data.shape[0])]
print("Finished running tokens")
white_data['lemmas'] = [[word.lemma_ for word in white_data['Patient_Summary_sp'].iloc[i]] for i in range(white_data.shape[0])]
black_data['lemmas'] = [[word.lemma_ for word in black_data['Patient_Summary_sp'].iloc[i]] for i in range(black_data.shape[0])]
mixed_data['lemmas'] = [[word.lemma_ for word in mixed_data['Patient_Summary_sp'].iloc[i]] for i in range(mixed_data.shape[0])]
asian_data['lemmas'] = [[word.lemma_ for word in asian_data['Patient_Summary_sp'].iloc[i]] for i in range(asian_data.shape[0])]
print("Finished running lemmas")

Finished running spacy
Finished running tokens
Finished running lemmas


In [24]:
for i in range(white_data.shape[0]):
    white_data['lemmas'].iloc[i] = [t for t in white_data['lemmas'].iloc[i] if t.isalnum()]
for i in range(black_data.shape[0]):
    black_data['lemmas'].iloc[i] = [t for t in black_data['lemmas'].iloc[i] if t.isalnum()]
for i in range(mixed_data.shape[0]):
    mixed_data['lemmas'].iloc[i] = [t for t in mixed_data['lemmas'].iloc[i] if t.isalnum()]
for i in range(asian_data.shape[0]):
    asian_data['lemmas'].iloc[i] = [t for t in asian_data['lemmas'].iloc[i] if t.isalnum()]

In [25]:
stop_words = set(stopwords.words('english'))
white_data["lemmas_no_stop"] = ''
black_data['lemmas_no_stop'] = ''
mixed_data['lemmas_no_stop'] = ''
asian_data['lemmas_no_stop'] = ''
for i in range(white_data.shape[0]):
    white_data['lemmas_no_stop'].iloc[i] = [t for t in white_data['lemmas'].iloc[i] if t.lower() not in stop_words and not t.isnumeric() and t.lower() not in ["i", "the", "SLaM", "i'm", "i’m", "zzzzz", "qqqqq"]]
for i in range(black_data.shape[0]):
    black_data['lemmas_no_stop'].iloc[i] = [t for t in black_data['lemmas'].iloc[i] if t.lower() not in stop_words and not t.isnumeric() and t.lower() not in ["i", "the", "SLaM", "i'm", "i’m", "zzzzz", "qqqqq"]]
for i in range(mixed_data.shape[0]):
    mixed_data['lemmas_no_stop'].iloc[i] = [t for t in mixed_data['lemmas'].iloc[i] if t.lower() not in stop_words and not t.isnumeric() and t.lower() not in ["i", "the", "SLaM", "i'm", "i’m", "zzzzz", "qqqqq"]]
for i in range(asian_data.shape[0]):
    asian_data['lemmas_no_stop'].iloc[i] = [t for t in asian_data['lemmas'].iloc[i] if t.lower() not in stop_words and not t.isnumeric() and t.lower() not in ["i", "the", "SLaM", "i'm", "i’m", "zzzzz", "qqqqq"]]

print("FINISHED REMOVING STOPWORDS")

FINISHED REMOVING STOPWORDS


In [ ]:
white_data.to_csv(index=False)
black_data.to_csv(index=False)
mixed_data.to_csv(index=False)
asian_data.to_csv(index=False)

In [9]:
def print_vocab(model, top_n = None):
    count = 0
    if top_n is not None:
        for index, word in enumerate(model.wv.index_to_key):
            count+= 1
            if count < top_n:
                print(f"WORD #{index}/{len(model.wv.index_to_key)} IS: {word}")

In [ ]:
display(white_data)

In [96]:
white_cbow_models = {}
# 100 dimensions
cbow_w3_f1_100 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, epochs = 10)
white_cbow_models["cbow_w3_f1_100"] = cbow_w3_f1_100
cbow_w5_f1_100 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, epochs = 10)
white_cbow_models["cbow_w5_f1_100"] = cbow_w5_f1_100
cbow_w10_f1_100 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, epochs = 10)
white_cbow_models["cbow_w10_f1_100"] = cbow_w10_f1_100
cbow_w3_f1_100_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 0, epochs = 10)
white_cbow_models["cbow_w3_f1_100_mc0"] = cbow_w3_f1_100_mc0
cbow_w5_f1_100_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 0, epochs = 10)
white_cbow_models["cbow_w5_f1_100_mc0"] = cbow_w5_f1_100_mc0
cbow_w10_f1_100_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 0, epochs = 10)
white_cbow_models["cbow_w10_f1_100_mc0"] = cbow_w10_f1_100_mc0
cbow_w10_f1_100_mc2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 0, epochs = 10)
white_cbow_models["cbow_w10_f1_100_mc2"] = cbow_w10_f1_100_mc2
cbow_w10_f1_100_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_100_neg2"] = cbow_w10_f1_100_neg2
cbow_w10_f1_100_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_100_neg5"] = cbow_w10_f1_100_neg5
cbow_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
white_cbow_models["cbow_w10_f1_100_ns_half_neg2"] = cbow_w10_f1_100_ns_half_neg2
cbow_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
white_cbow_models["cbow_w10_f1_100_ns_half_neg3"] = cbow_w10_f1_100_ns_half_neg3
cbow_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
white_cbow_models["cbow_w10_f1_100_ns_half_neg5"] = cbow_w10_f1_100_ns_half_neg5
cbow_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_100_neg2_ws2"] = cbow_w10_f1_100_neg2_ws2
cbow_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_100_neg5_ws5"] = cbow_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION MODELS---WHITE")

# 300 dimensions
cbow_w3_f1_300 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, epochs = 10)
white_cbow_models["cbow_w3_f1_300"] = cbow_w3_f1_300
cbow_w5_f1_300 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, epochs = 10)
white_cbow_models["cbow_w5_f1_300"] = cbow_w5_f1_300
cbow_w10_f1_300 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, epochs = 10)
white_cbow_models["cbow_w10_f1_300"] = cbow_w10_f1_300
cbow_w3_f1_300_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 0, epochs = 10)
white_cbow_models["cbow_w3_f1_300_mc0"] = cbow_w3_f1_300_mc0
cbow_w5_f1_300_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 0, epochs = 10)
white_cbow_models["cbow_w5_f1_300_mc0"] = cbow_w5_f1_300_mc0
cbow_w10_f1_300_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 0, epochs = 10)
white_cbow_models["cbow_w10_f1_300_mc0"] = cbow_w10_f1_300_mc0
cbow_w10_f1_300_mc2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 0, epochs = 10)
white_cbow_models["cbow_w10_f1_300_mc2"] = cbow_w10_f1_300_mc2
cbow_w10_f1_300_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_300_neg2"] = cbow_w10_f1_300_neg2
cbow_w10_f1_300_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_300_neg5"] = cbow_w10_f1_300_neg5
cbow_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
white_cbow_models["cbow_w10_f1_300_ns_half_neg2"] = cbow_w10_f1_300_ns_half_neg2
cbow_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
white_cbow_models["cbow_w10_f1_300_ns_half_neg3"] = cbow_w10_f1_300_ns_half_neg3
cbow_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
white_cbow_models["cbow_w10_f1_300_ns_half_neg5"] = cbow_w10_f1_300_ns_half_neg5
cbow_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_300_neg2_ws2"] = cbow_w10_f1_300_neg2_ws2
cbow_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_300_neg5_ws5"] = cbow_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION MODELS---WHITE")

# 500 dimensions
cbow_w3_f1_500 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, epochs = 10)
white_cbow_models["cbow_w3_f1_500"] = cbow_w3_f1_500
cbow_w5_f1_500 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, epochs = 10)
white_cbow_models["cbow_w5_f1_500"] = cbow_w5_f1_500
cbow_w10_f1_500 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, epochs = 10)
white_cbow_models["cbow_w10_f1_500"] = cbow_w10_f1_500
cbow_w3_f1_500_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 0, epochs = 10)
white_cbow_models["cbow_w3_f1_500_mc0"] = cbow_w3_f1_500_mc0
cbow_w5_f1_500_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 0, epochs = 10)
white_cbow_models["cbow_w5_f1_500_mc0"] = cbow_w5_f1_500_mc0
cbow_w10_f1_500_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 0, epochs = 10)
white_cbow_models["cbow_w10_f1_500_mc0"] = cbow_w10_f1_500_mc0
cbow_w10_f1_500_mc2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 0, epochs = 10)
white_cbow_models["cbow_w10_f1_500_mc2"] = cbow_w10_f1_500_mc2
cbow_w10_f1_500_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_500_neg2"] = cbow_w10_f1_500_neg2
cbow_w10_f1_500_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_500_neg5"] = cbow_w10_f1_500_neg5
cbow_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
white_cbow_models["cbow_w10_f1_500_ns_half_neg2"] = cbow_w10_f1_500_ns_half_neg2
cbow_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
white_cbow_models["cbow_w10_f1_500_ns_half_neg3"] = cbow_w10_f1_500_ns_half_neg3
cbow_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
white_cbow_models["cbow_w10_f1_500_ns_half_neg5"] = cbow_w10_f1_500_ns_half_neg5
cbow_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_500_neg2_ws2"] = cbow_w10_f1_500_neg2_ws2
cbow_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_500_neg5_ws5"] = cbow_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION MODELS---WHITE")

# 700 dimensions
cbow_w3_f1_700 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, epochs = 10)
white_cbow_models["cbow_w3_f1_700"] = cbow_w3_f1_700
cbow_w5_f1_700 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, epochs = 10)
white_cbow_models["cbow_w5_f1_700"] = cbow_w5_f1_700
cbow_w10_f1_700 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, epochs = 10)
white_cbow_models["cbow_w10_f1_700"] = cbow_w10_f1_700
cbow_w3_f1_700_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 0, epochs = 10)
white_cbow_models["cbow_w3_f1_700_mc0"] = cbow_w3_f1_700_mc0
cbow_w5_f1_700_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 0, epochs = 10)
white_cbow_models["cbow_w5_f1_700_mc0"] = cbow_w5_f1_700_mc0
cbow_w10_f1_700_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 0, epochs = 10)
white_cbow_models["cbow_w10_f1_700_mc0"] = cbow_w10_f1_700_mc0
cbow_w10_f1_700_mc2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 0, epochs = 10)
white_cbow_models["cbow_w10_f1_700_mc2"] = cbow_w10_f1_700_mc2
cbow_w10_f1_700_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_700_neg2"] = cbow_w10_f1_700_neg2
cbow_w10_f1_700_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_700_neg5"] = cbow_w10_f1_700_neg5
cbow_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
white_cbow_models["cbow_w10_f1_700_ns_half_neg2"] = cbow_w10_f1_700_ns_half_neg2
cbow_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
white_cbow_models["cbow_w10_f1_700_ns_half_neg3"] = cbow_w10_f1_700_ns_half_neg3
cbow_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
white_cbow_models["cbow_w10_f1_700_ns_half_neg5"] = cbow_w10_f1_700_ns_half_neg5
cbow_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_700_neg2_ws2"] = cbow_w10_f1_700_neg2_ws2
cbow_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_700_neg5_ws5"] = cbow_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION MODELS---WHITE")

# 1000 dimensions
cbow_w3_f1_1000 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
white_cbow_models["cbow_w3_f1_1000"] = cbow_w3_f1_1000
cbow_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w2_f1_1000_ns_half_neg5"] = cbow_w2_f1_1000_ns_half_neg5
cbow_w5_f1_1000 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, epochs = 10)
white_cbow_models["cbow_w5_f1_1000"] = cbow_w5_f1_1000
cbow_w10_f1_1000 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, epochs = 10)
white_cbow_models["cbow_w10_f1_1000"] = cbow_w10_f1_1000
cbow_w3_f1_1000_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 10)
white_cbow_models["cbow_w3_f1_1000_mc0"] = cbow_w3_f1_1000_mc0

cbow_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 20)
white_cbow_models["cbow_w3_f1_1000_mc0_e20"] = cbow_w3_f1_1000_mc0_e20

cbow_w3_f1_1000_mc1 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
white_cbow_models["cbow_w3_f1_1000_mc1"] = cbow_w3_f1_1000_mc1

cbow_w5_f1_1000_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 0, epochs = 10)
white_cbow_models["cbow_w5_f1_1000_mc0"] = cbow_w5_f1_1000_mc0
cbow_w10_f1_1000_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 0, epochs = 10)
white_cbow_models["cbow_w10_f1_1000_mc0"] = cbow_w10_f1_1000_mc0
cbow_w10_f1_1000_mc2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 0, epochs = 10)
white_cbow_models["cbow_w10_f1_1000_mc2"] = cbow_w10_f1_1000_mc2
cbow_w10_f1_1000_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_1000_neg2"] = cbow_w10_f1_1000_neg2
cbow_w10_f1_1000_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_1000_neg5"] = cbow_w10_f1_1000_neg5
cbow_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
white_cbow_models["cbow_w10_f1_1000_ns_half_neg2"] = cbow_w10_f1_1000_ns_half_neg2
cbow_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
white_cbow_models["cbow_w10_f1_1000_ns_half_neg3"] = cbow_w10_f1_1000_ns_half_neg3
cbow_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
white_cbow_models["cbow_w10_f1_1000_ns_half_neg5"] = cbow_w10_f1_1000_ns_half_neg5
cbow_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_1000_neg2_ws2"] = cbow_w10_f1_1000_neg2_ws2
cbow_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_cbow_models["cbow_w10_f1_1000_neg5_ws5"] = cbow_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION MODELS---WHITE")

TRAINED ALL 100-DIMENSION MODELS---WHITE
TRAINED ALL 300-DIMENSION MODELS---WHITE
TRAINED ALL 500-DIMENSION MODELS---WHITE
TRAINED ALL 700-DIMENSION MODELS---WHITE
TRAINED ALL 1000-DIMENSION MODELS---WHITE


In [99]:
# white cbow save & calculate avg
synonyms = dict({"medication": "medicine",
                 "therapy": "treatment",
                 "simple": "easy",
                 "appointment": "engagement",
                 "road": "route",
                 "family": "household",
                 "history": "account",
                 "mood": "temper",
                 "information": "info",
                 "treatment": "intervention",
                 "social": "sociable",
                 "anxious": "nervous",
                 "confidential": "private",
                 "feeling": "belief",
                 "feeling": "opinion",
                 "feeling": "impression",
                 "aware": "mindful",
                 "difficult": "hard",
                 "clinician": "doctor",
                 "clinician": "therapist"
                })

white_cbow_models_synonymity_average = {}
white_cbow_names = list(white_cbow_models.keys())
white_cbow_vals = list(white_cbow_models.values())
for i in range(len(white_cbow_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(white_cbow_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    white_cbow_models_synonymity_average[white_cbow_names[i]] = average_synonimity

for i in white_cbow_models_synonymity_average.keys():
    print(f"{i}: {white_cbow_models_synonymity_average[i]}")
white_cbow_model_chosen = max(white_cbow_models_synonymity_average, key=white_cbow_models_synonymity_average.get)
print(f"WHITE CBOW MODEL: {white_cbow_model_chosen}")
print(f"\tSYNONIMITY: {white_cbow_models_synonymity_average[white_cbow_model_chosen]}")
white_cbow_models[white_cbow_model_chosen].save(os.path.join('white_models', white_cbow_model_chosen))

cbow_w3_f1_100: 0.7611773610115051
cbow_w5_f1_100: 0.7406727075576782
cbow_w10_f1_100: 0.5874452590942383
cbow_w3_f1_100_mc0: 0.7618361711502075
cbow_w5_f1_100_mc0: 0.7310590744018555
cbow_w10_f1_100_mc0: 0.5897625684738159
cbow_w10_f1_100_mc2: 0.5358892679214478
cbow_w10_f1_100_neg2: 0.6089947819709778
cbow_w10_f1_100_neg5: 0.5832439064979553
cbow_w10_f1_100_ns_half_neg2: 0.730128288269043
cbow_w10_f1_100_ns_half_neg3: 0.7315319776535034
cbow_w10_f1_100_ns_half_neg5: 0.6801722645759583
cbow_w10_f1_100_neg2_ws2: 0.7331801056861877
cbow_w10_f1_100_neg5_ws5: 0.7454401254653931
cbow_w3_f1_300: 0.7681673765182495
cbow_w5_f1_300: 0.7526143789291382
cbow_w10_f1_300: 0.6059569716453552
cbow_w3_f1_300_mc0: 0.781477153301239
cbow_w5_f1_300_mc0: 0.7532844543457031
cbow_w10_f1_300_mc0: 0.6045530438423157
cbow_w10_f1_300_mc2: 0.5557360053062439
cbow_w10_f1_300_neg2: 0.6129350066184998
cbow_w10_f1_300_neg5: 0.6120279431343079
cbow_w10_f1_300_ns_half_neg2: 0.7666320204734802
cbow_w10_f1_300_ns_half_

In [100]:
black_cbow_models = {}
# 100 dimensions
cbow_w3_f1_100 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, epochs = 10)
black_cbow_models["cbow_w3_f1_100"] = cbow_w3_f1_100
cbow_w5_f1_100 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, epochs = 10)
black_cbow_models["cbow_w5_f1_100"] = cbow_w5_f1_100
cbow_w10_f1_100 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, epochs = 10)
black_cbow_models["cbow_w10_f1_100"] = cbow_w10_f1_100
cbow_w3_f1_100_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 0, epochs = 10)
black_cbow_models["cbow_w3_f1_100_mc0"] = cbow_w3_f1_100_mc0
cbow_w5_f1_100_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 0, epochs = 10)
black_cbow_models["cbow_w5_f1_100_mc0"] = cbow_w5_f1_100_mc0
cbow_w10_f1_100_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 0, epochs = 10)
black_cbow_models["cbow_w10_f1_100_mc0"] = cbow_w10_f1_100_mc0
cbow_w10_f1_100_mc2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 0, epochs = 10)
black_cbow_models["cbow_w10_f1_100_mc2"] = cbow_w10_f1_100_mc2
cbow_w10_f1_100_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_100_neg2"] = cbow_w10_f1_100_neg2
cbow_w10_f1_100_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_100_neg5"] = cbow_w10_f1_100_neg5
cbow_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
black_cbow_models["cbow_w10_f1_100_ns_half_neg2"] = cbow_w10_f1_100_ns_half_neg2
cbow_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
black_cbow_models["cbow_w10_f1_100_ns_half_neg3"] = cbow_w10_f1_100_ns_half_neg3
cbow_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
black_cbow_models["cbow_w10_f1_100_ns_half_neg5"] = cbow_w10_f1_100_ns_half_neg5
cbow_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_100_neg2_ws2"] = cbow_w10_f1_100_neg2_ws2
cbow_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_100_neg5_ws5"] = cbow_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION MODELS---BLACK")

# 300 dimensions
cbow_w3_f1_300 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, epochs = 10)
black_cbow_models["cbow_w3_f1_300"] = cbow_w3_f1_300
cbow_w5_f1_300 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, epochs = 10)
black_cbow_models["cbow_w5_f1_300"] = cbow_w5_f1_300
cbow_w10_f1_300 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, epochs = 10)
black_cbow_models["cbow_w10_f1_300"] = cbow_w10_f1_300
cbow_w3_f1_300_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 0, epochs = 10)
black_cbow_models["cbow_w3_f1_300_mc0"] = cbow_w3_f1_300_mc0
cbow_w5_f1_300_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 0, epochs = 10)
black_cbow_models["cbow_w5_f1_300_mc0"] = cbow_w5_f1_300_mc0
cbow_w10_f1_300_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 0, epochs = 10)
black_cbow_models["cbow_w10_f1_300_mc0"] = cbow_w10_f1_300_mc0
cbow_w10_f1_300_mc2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 0, epochs = 10)
black_cbow_models["cbow_w10_f1_300_mc2"] = cbow_w10_f1_300_mc2
cbow_w10_f1_300_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_300_neg2"] = cbow_w10_f1_300_neg2
cbow_w10_f1_300_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_300_neg5"] = cbow_w10_f1_300_neg5
cbow_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
black_cbow_models["cbow_w10_f1_300_ns_half_neg2"] = cbow_w10_f1_300_ns_half_neg2
cbow_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
black_cbow_models["cbow_w10_f1_300_ns_half_neg3"] = cbow_w10_f1_300_ns_half_neg3
cbow_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
black_cbow_models["cbow_w10_f1_300_ns_half_neg5"] = cbow_w10_f1_300_ns_half_neg5
cbow_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_300_neg2_ws2"] = cbow_w10_f1_300_neg2_ws2
cbow_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_300_neg5_ws5"] = cbow_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION MODELS---BLACK")

# 500 dimensions
cbow_w3_f1_500 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, epochs = 10)
black_cbow_models["cbow_w3_f1_500"] = cbow_w3_f1_500
cbow_w5_f1_500 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, epochs = 10)
black_cbow_models["cbow_w5_f1_500"] = cbow_w5_f1_500
cbow_w10_f1_500 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, epochs = 10)
black_cbow_models["cbow_w10_f1_500"] = cbow_w10_f1_500
cbow_w3_f1_500_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 0, epochs = 10)
black_cbow_models["cbow_w3_f1_500_mc0"] = cbow_w3_f1_500_mc0
cbow_w5_f1_500_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 0, epochs = 10)
black_cbow_models["cbow_w5_f1_500_mc0"] = cbow_w5_f1_500_mc0
cbow_w10_f1_500_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 0, epochs = 10)
black_cbow_models["cbow_w10_f1_500_mc0"] = cbow_w10_f1_500_mc0
cbow_w10_f1_500_mc2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 0, epochs = 10)
black_cbow_models["cbow_w10_f1_500_mc2"] = cbow_w10_f1_500_mc2
cbow_w10_f1_500_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_500_neg2"] = cbow_w10_f1_500_neg2
cbow_w10_f1_500_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_500_neg5"] = cbow_w10_f1_500_neg5
cbow_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
black_cbow_models["cbow_w10_f1_500_ns_half_neg2"] = cbow_w10_f1_500_ns_half_neg2
cbow_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
black_cbow_models["cbow_w10_f1_500_ns_half_neg3"] = cbow_w10_f1_500_ns_half_neg3
cbow_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
black_cbow_models["cbow_w10_f1_500_ns_half_neg5"] = cbow_w10_f1_500_ns_half_neg5
cbow_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_500_neg2_ws2"] = cbow_w10_f1_500_neg2_ws2
cbow_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_500_neg5_ws5"] = cbow_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION MODELS---BLACK")

# 700 dimensions
cbow_w3_f1_700 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, epochs = 10)
black_cbow_models["cbow_w3_f1_700"] = cbow_w3_f1_700
cbow_w5_f1_700 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, epochs = 10)
black_cbow_models["cbow_w5_f1_700"] = cbow_w5_f1_700
cbow_w10_f1_700 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, epochs = 10)
black_cbow_models["cbow_w10_f1_700"] = cbow_w10_f1_700
cbow_w3_f1_700_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 0, epochs = 10)
black_cbow_models["cbow_w3_f1_700_mc0"] = cbow_w3_f1_700_mc0
cbow_w5_f1_700_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 0, epochs = 10)
black_cbow_models["cbow_w5_f1_700_mc0"] = cbow_w5_f1_700_mc0
cbow_w10_f1_700_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 0, epochs = 10)
black_cbow_models["cbow_w10_f1_700_mc0"] = cbow_w10_f1_700_mc0
cbow_w10_f1_700_mc2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 0, epochs = 10)
black_cbow_models["cbow_w10_f1_700_mc2"] = cbow_w10_f1_700_mc2
cbow_w10_f1_700_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_700_neg2"] = cbow_w10_f1_700_neg2
cbow_w10_f1_700_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_700_neg5"] = cbow_w10_f1_700_neg5
cbow_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
black_cbow_models["cbow_w10_f1_700_ns_half_neg2"] = cbow_w10_f1_700_ns_half_neg2
cbow_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
black_cbow_models["cbow_w10_f1_700_ns_half_neg3"] = cbow_w10_f1_700_ns_half_neg3
cbow_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
black_cbow_models["cbow_w10_f1_700_ns_half_neg5"] = cbow_w10_f1_700_ns_half_neg5
cbow_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_700_neg2_ws2"] = cbow_w10_f1_700_neg2_ws2
cbow_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_700_neg5_ws5"] = cbow_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION MODELS---BLACK")

# 1000 dimensions
cbow_w3_f1_1000 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
black_cbow_models["cbow_w3_f1_1000"] = cbow_w3_f1_1000
cbow_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w2_f1_1000_ns_half_neg5"] = cbow_w2_f1_1000_ns_half_neg5
cbow_w5_f1_1000 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, epochs = 10)
black_cbow_models["cbow_w5_f1_1000"] = cbow_w5_f1_1000
cbow_w10_f1_1000 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, epochs = 10)
black_cbow_models["cbow_w10_f1_1000"] = cbow_w10_f1_1000
cbow_w3_f1_1000_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 10)
black_cbow_models["cbow_w3_f1_1000_mc0"] = cbow_w3_f1_1000_mc0

cbow_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 20)
black_cbow_models["cbow_w3_f1_1000_mc0_e20"] = cbow_w3_f1_1000_mc0_e20

cbow_w3_f1_1000_mc1 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
black_cbow_models["cbow_w3_f1_1000_mc1"] = cbow_w3_f1_1000_mc1

cbow_w5_f1_1000_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 0, epochs = 10)
black_cbow_models["cbow_w5_f1_1000_mc0"] = cbow_w5_f1_1000_mc0
cbow_w10_f1_1000_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 0, epochs = 10)
black_cbow_models["cbow_w10_f1_1000_mc0"] = cbow_w10_f1_1000_mc0
cbow_w10_f1_1000_mc2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 0, epochs = 10)
black_cbow_models["cbow_w10_f1_1000_mc2"] = cbow_w10_f1_1000_mc2
cbow_w10_f1_1000_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_1000_neg2"] = cbow_w10_f1_1000_neg2
cbow_w10_f1_1000_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_1000_neg5"] = cbow_w10_f1_1000_neg5
cbow_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
black_cbow_models["cbow_w10_f1_1000_ns_half_neg2"] = cbow_w10_f1_1000_ns_half_neg2
cbow_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
black_cbow_models["cbow_w10_f1_1000_ns_half_neg3"] = cbow_w10_f1_1000_ns_half_neg3
cbow_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
black_cbow_models["cbow_w10_f1_1000_ns_half_neg5"] = cbow_w10_f1_1000_ns_half_neg5
cbow_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_1000_neg2_ws2"] = cbow_w10_f1_1000_neg2_ws2
cbow_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_cbow_models["cbow_w10_f1_1000_neg5_ws5"] = cbow_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION MODELS---BLACK")

TRAINED ALL 100-DIMENSION MODELS---BLACK
TRAINED ALL 300-DIMENSION MODELS---BLACK
TRAINED ALL 500-DIMENSION MODELS---BLACK
TRAINED ALL 700-DIMENSION MODELS---BLACK
TRAINED ALL 1000-DIMENSION MODELS---BLACK


In [101]:
# black cbow save & calculate avg
# synonyms = dict({"appointment": "engagement",
#                  "road": "route",
#                  "family": "household",
#                  "history": "account",
#                  "mood": "temper",
#                  "treatment": "intervention",
#                  "confidential": "private",
#                  "feeling": "belief",
#                  "feeling": "opinion",
#                  "feeling": "impression",
#                 })
synonyms = dict({"appointment": "engagement",
                 "family": "household",
                 "history": "account",
                 "treatment": "intervention",
                 "difficult": "difficulty",
                 "clinician": "doctor",
                 "clinician": "therapist",
                })
black_cbow_models_synonymity_average = dict()
black_cbow_names = list(black_cbow_models.keys())
black_cbow_vals = list(black_cbow_models.values())
for i in range(len(black_cbow_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(black_cbow_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    black_cbow_models_synonymity_average[black_cbow_names[i]] = average_synonimity

black_cbow_model_chosen = max(black_cbow_models_synonymity_average, key=black_cbow_models_synonymity_average.get)
print(f"BLACK CBOW MODEL: {black_cbow_model_chosen}")
print(f"\tSYNONIMITY: {black_cbow_models_synonymity_average[black_cbow_model_chosen]}")
black_cbow_models[black_cbow_model_chosen].save(os.path.join('black_models', black_cbow_model_chosen))

BLACK CBOW MODEL: cbow_w10_f1_100_ns_half_neg2
	SYNONIMITY: 0.9570443630218506


In [102]:
mixed_cbow_models = {}
# 100 dimensions
cbow_w3_f1_100 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w3_f1_100"] = cbow_w3_f1_100
cbow_w5_f1_100 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w5_f1_100"] = cbow_w5_f1_100
cbow_w10_f1_100 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w10_f1_100"] = cbow_w10_f1_100
cbow_w3_f1_100_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w3_f1_100_mc0"] = cbow_w3_f1_100_mc0
cbow_w5_f1_100_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w5_f1_100_mc0"] = cbow_w5_f1_100_mc0
cbow_w10_f1_100_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w10_f1_100_mc0"] = cbow_w10_f1_100_mc0
cbow_w10_f1_100_mc2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w10_f1_100_mc2"] = cbow_w10_f1_100_mc2
cbow_w10_f1_100_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_100_neg2"] = cbow_w10_f1_100_neg2
cbow_w10_f1_100_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_100_neg5"] = cbow_w10_f1_100_neg5
cbow_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
mixed_cbow_models["cbow_w10_f1_100_ns_half_neg2"] = cbow_w10_f1_100_ns_half_neg2
cbow_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
mixed_cbow_models["cbow_w10_f1_100_ns_half_neg3"] = cbow_w10_f1_100_ns_half_neg3
cbow_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
mixed_cbow_models["cbow_w10_f1_100_ns_half_neg5"] = cbow_w10_f1_100_ns_half_neg5
cbow_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_100_neg2_ws2"] = cbow_w10_f1_100_neg2_ws2
cbow_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_100_neg5_ws5"] = cbow_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION MODELS---MIXED")

# 300 dimensions
cbow_w3_f1_300 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w3_f1_300"] = cbow_w3_f1_300
cbow_w5_f1_300 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w5_f1_300"] = cbow_w5_f1_300
cbow_w10_f1_300 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w10_f1_300"] = cbow_w10_f1_300
cbow_w3_f1_300_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w3_f1_300_mc0"] = cbow_w3_f1_300_mc0
cbow_w5_f1_300_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w5_f1_300_mc0"] = cbow_w5_f1_300_mc0
cbow_w10_f1_300_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w10_f1_300_mc0"] = cbow_w10_f1_300_mc0
cbow_w10_f1_300_mc2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w10_f1_300_mc2"] = cbow_w10_f1_300_mc2
cbow_w10_f1_300_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_300_neg2"] = cbow_w10_f1_300_neg2
cbow_w10_f1_300_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_300_neg5"] = cbow_w10_f1_300_neg5
cbow_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
mixed_cbow_models["cbow_w10_f1_300_ns_half_neg2"] = cbow_w10_f1_300_ns_half_neg2
cbow_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
mixed_cbow_models["cbow_w10_f1_300_ns_half_neg3"] = cbow_w10_f1_300_ns_half_neg3
cbow_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
mixed_cbow_models["cbow_w10_f1_300_ns_half_neg5"] = cbow_w10_f1_300_ns_half_neg5
cbow_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_300_neg2_ws2"] = cbow_w10_f1_300_neg2_ws2
cbow_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_300_neg5_ws5"] = cbow_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION MODELS---MIXED")

# 500 dimensions
cbow_w3_f1_500 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w3_f1_500"] = cbow_w3_f1_500
cbow_w5_f1_500 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w5_f1_500"] = cbow_w5_f1_500
cbow_w10_f1_500 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w10_f1_500"] = cbow_w10_f1_500
cbow_w3_f1_500_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w3_f1_500_mc0"] = cbow_w3_f1_500_mc0
cbow_w5_f1_500_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w5_f1_500_mc0"] = cbow_w5_f1_500_mc0
cbow_w10_f1_500_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w10_f1_500_mc0"] = cbow_w10_f1_500_mc0
cbow_w10_f1_500_mc2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w10_f1_500_mc2"] = cbow_w10_f1_500_mc2
cbow_w10_f1_500_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_500_neg2"] = cbow_w10_f1_500_neg2
cbow_w10_f1_500_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_500_neg5"] = cbow_w10_f1_500_neg5
cbow_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
mixed_cbow_models["cbow_w10_f1_500_ns_half_neg2"] = cbow_w10_f1_500_ns_half_neg2
cbow_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
mixed_cbow_models["cbow_w10_f1_500_ns_half_neg3"] = cbow_w10_f1_500_ns_half_neg3
cbow_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
mixed_cbow_models["cbow_w10_f1_500_ns_half_neg5"] = cbow_w10_f1_500_ns_half_neg5
cbow_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_500_neg2_ws2"] = cbow_w10_f1_500_neg2_ws2
cbow_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_500_neg5_ws5"] = cbow_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION MODELS---MIXED")

# 700 dimensions
cbow_w3_f1_700 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w3_f1_700"] = cbow_w3_f1_700
cbow_w5_f1_700 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w5_f1_700"] = cbow_w5_f1_700
cbow_w10_f1_700 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w10_f1_700"] = cbow_w10_f1_700
cbow_w3_f1_700_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w3_f1_700_mc0"] = cbow_w3_f1_700_mc0
cbow_w5_f1_700_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w5_f1_700_mc0"] = cbow_w5_f1_700_mc0
cbow_w10_f1_700_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w10_f1_700_mc0"] = cbow_w10_f1_700_mc0
cbow_w10_f1_700_mc2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w10_f1_700_mc2"] = cbow_w10_f1_700_mc2
cbow_w10_f1_700_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_700_neg2"] = cbow_w10_f1_700_neg2
cbow_w10_f1_700_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_700_neg5"] = cbow_w10_f1_700_neg5
cbow_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
mixed_cbow_models["cbow_w10_f1_700_ns_half_neg2"] = cbow_w10_f1_700_ns_half_neg2
cbow_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
mixed_cbow_models["cbow_w10_f1_700_ns_half_neg3"] = cbow_w10_f1_700_ns_half_neg3
cbow_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
mixed_cbow_models["cbow_w10_f1_700_ns_half_neg5"] = cbow_w10_f1_700_ns_half_neg5
cbow_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_700_neg2_ws2"] = cbow_w10_f1_700_neg2_ws2
cbow_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_700_neg5_ws5"] = cbow_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION MODELS---MIXED")

# 1000 dimensions
cbow_w3_f1_1000 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w3_f1_1000"] = cbow_w3_f1_1000
cbow_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w2_f1_1000_ns_half_neg5"] = cbow_w2_f1_1000_ns_half_neg5
cbow_w5_f1_1000 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w5_f1_1000"] = cbow_w5_f1_1000
cbow_w10_f1_1000 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w10_f1_1000"] = cbow_w10_f1_1000
cbow_w3_f1_1000_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w3_f1_1000_mc0"] = cbow_w3_f1_1000_mc0

cbow_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 20)
mixed_cbow_models["cbow_w3_f1_1000_mc0_e20"] = cbow_w3_f1_1000_mc0_e20

cbow_w3_f1_1000_mc1 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w3_f1_1000_mc1"] = cbow_w3_f1_1000_mc1

cbow_w5_f1_1000_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w5_f1_1000_mc0"] = cbow_w5_f1_1000_mc0
cbow_w10_f1_1000_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w10_f1_1000_mc0"] = cbow_w10_f1_1000_mc0
cbow_w10_f1_1000_mc2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 0, epochs = 10)
mixed_cbow_models["cbow_w10_f1_1000_mc2"] = cbow_w10_f1_1000_mc2
cbow_w10_f1_1000_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_1000_neg2"] = cbow_w10_f1_1000_neg2
cbow_w10_f1_1000_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_1000_neg5"] = cbow_w10_f1_1000_neg5
cbow_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
mixed_cbow_models["cbow_w10_f1_1000_ns_half_neg2"] = cbow_w10_f1_1000_ns_half_neg2
cbow_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
mixed_cbow_models["cbow_w10_f1_1000_ns_half_neg3"] = cbow_w10_f1_1000_ns_half_neg3
cbow_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
mixed_cbow_models["cbow_w10_f1_1000_ns_half_neg5"] = cbow_w10_f1_1000_ns_half_neg5
cbow_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_1000_neg2_ws2"] = cbow_w10_f1_1000_neg2_ws2
cbow_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_cbow_models["cbow_w10_f1_1000_neg5_ws5"] = cbow_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION MODELS---MIXED")

TRAINED ALL 100-DIMENSION MODELS---MIXED
TRAINED ALL 300-DIMENSION MODELS---MIXED
TRAINED ALL 500-DIMENSION MODELS---MIXED
TRAINED ALL 700-DIMENSION MODELS---MIXED
TRAINED ALL 1000-DIMENSION MODELS---MIXED


In [105]:
# mixed cbow save & calculate avg
synonyms = dict({"appointment": "engagement",
                 "family": "household",
                 "history": "account",
                 "treatment": "intervention",
                 "difficult": "difficulty",
                })

mixed_cbow_models_synonymity_average = dict()
mixed_cbow_names = list(mixed_cbow_models.keys())
mixed_cbow_vals = list(mixed_cbow_models.values())
for i in range(len(mixed_cbow_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(mixed_cbow_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    mixed_cbow_models_synonymity_average[mixed_cbow_names[i]] = average_synonimity

mixed_cbow_model_chosen = max(mixed_cbow_models_synonymity_average, key=mixed_cbow_models_synonymity_average.get)
print(f"MIXED CBOW MODEL: {mixed_cbow_model_chosen}")
print(f"\tSYNONIMITY: {mixed_cbow_models_synonymity_average[mixed_cbow_model_chosen]}")
mixed_cbow_models[mixed_cbow_model_chosen].save(os.path.join('mixed_models', mixed_cbow_model_chosen))

MIXED CBOW MODEL: cbow_w10_f1_1000_mc2
	SYNONIMITY: 0.9995271563529968


In [106]:
asian_cbow_models = {}
# 100 dimensions
cbow_w3_f1_100 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, epochs = 10)
asian_cbow_models["cbow_w3_f1_100"] = cbow_w3_f1_100
cbow_w5_f1_100 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, epochs = 10)
asian_cbow_models["cbow_w5_f1_100"] = cbow_w5_f1_100
cbow_w10_f1_100 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, epochs = 10)
asian_cbow_models["cbow_w10_f1_100"] = cbow_w10_f1_100
cbow_w3_f1_100_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 0, epochs = 10)
asian_cbow_models["cbow_w3_f1_100_mc0"] = cbow_w3_f1_100_mc0
cbow_w5_f1_100_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 0, epochs = 10)
asian_cbow_models["cbow_w5_f1_100_mc0"] = cbow_w5_f1_100_mc0
cbow_w10_f1_100_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 0, epochs = 10)
asian_cbow_models["cbow_w10_f1_100_mc0"] = cbow_w10_f1_100_mc0
cbow_w10_f1_100_mc2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 0, epochs = 10)
asian_cbow_models["cbow_w10_f1_100_mc2"] = cbow_w10_f1_100_mc2
cbow_w10_f1_100_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_100_neg2"] = cbow_w10_f1_100_neg2
cbow_w10_f1_100_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_100_neg5"] = cbow_w10_f1_100_neg5
cbow_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
asian_cbow_models["cbow_w10_f1_100_ns_half_neg2"] = cbow_w10_f1_100_ns_half_neg2
cbow_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
asian_cbow_models["cbow_w10_f1_100_ns_half_neg3"] = cbow_w10_f1_100_ns_half_neg3
cbow_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
asian_cbow_models["cbow_w10_f1_100_ns_half_neg5"] = cbow_w10_f1_100_ns_half_neg5
cbow_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_100_neg2_ws2"] = cbow_w10_f1_100_neg2_ws2
cbow_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_100_neg5_ws5"] = cbow_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION MODELS---ASIAN")

# 300 dimensions
cbow_w3_f1_300 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, epochs = 10)
asian_cbow_models["cbow_w3_f1_300"] = cbow_w3_f1_300
cbow_w5_f1_300 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, epochs = 10)
asian_cbow_models["cbow_w5_f1_300"] = cbow_w5_f1_300
cbow_w10_f1_300 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, epochs = 10)
asian_cbow_models["cbow_w10_f1_300"] = cbow_w10_f1_300
cbow_w3_f1_300_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 0, epochs = 10)
asian_cbow_models["cbow_w3_f1_300_mc0"] = cbow_w3_f1_300_mc0
cbow_w5_f1_300_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 0, epochs = 10)
asian_cbow_models["cbow_w5_f1_300_mc0"] = cbow_w5_f1_300_mc0
cbow_w10_f1_300_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 0, epochs = 10)
asian_cbow_models["cbow_w10_f1_300_mc0"] = cbow_w10_f1_300_mc0
cbow_w10_f1_300_mc2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 0, epochs = 10)
asian_cbow_models["cbow_w10_f1_300_mc2"] = cbow_w10_f1_300_mc2
cbow_w10_f1_300_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_300_neg2"] = cbow_w10_f1_300_neg2
cbow_w10_f1_300_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_300_neg5"] = cbow_w10_f1_300_neg5
cbow_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
asian_cbow_models["cbow_w10_f1_300_ns_half_neg2"] = cbow_w10_f1_300_ns_half_neg2
cbow_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
asian_cbow_models["cbow_w10_f1_300_ns_half_neg3"] = cbow_w10_f1_300_ns_half_neg3
cbow_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
asian_cbow_models["cbow_w10_f1_300_ns_half_neg5"] = cbow_w10_f1_300_ns_half_neg5
cbow_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_300_neg2_ws2"] = cbow_w10_f1_300_neg2_ws2
cbow_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_300_neg5_ws5"] = cbow_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION MODELS---ASIAN")

# 500 dimensions
cbow_w3_f1_500 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, epochs = 10)
asian_cbow_models["cbow_w3_f1_500"] = cbow_w3_f1_500
cbow_w5_f1_500 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, epochs = 10)
asian_cbow_models["cbow_w5_f1_500"] = cbow_w5_f1_500
cbow_w10_f1_500 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, epochs = 10)
asian_cbow_models["cbow_w10_f1_500"] = cbow_w10_f1_500
cbow_w3_f1_500_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 0, epochs = 10)
asian_cbow_models["cbow_w3_f1_500_mc0"] = cbow_w3_f1_500_mc0
cbow_w5_f1_500_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 0, epochs = 10)
asian_cbow_models["cbow_w5_f1_500_mc0"] = cbow_w5_f1_500_mc0
cbow_w10_f1_500_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 0, epochs = 10)
asian_cbow_models["cbow_w10_f1_500_mc0"] = cbow_w10_f1_500_mc0
cbow_w10_f1_500_mc2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 0, epochs = 10)
asian_cbow_models["cbow_w10_f1_500_mc2"] = cbow_w10_f1_500_mc2
cbow_w10_f1_500_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_500_neg2"] = cbow_w10_f1_500_neg2
cbow_w10_f1_500_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_500_neg5"] = cbow_w10_f1_500_neg5
cbow_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
asian_cbow_models["cbow_w10_f1_500_ns_half_neg2"] = cbow_w10_f1_500_ns_half_neg2
cbow_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
asian_cbow_models["cbow_w10_f1_500_ns_half_neg3"] = cbow_w10_f1_500_ns_half_neg3
cbow_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
asian_cbow_models["cbow_w10_f1_500_ns_half_neg5"] = cbow_w10_f1_500_ns_half_neg5
cbow_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_500_neg2_ws2"] = cbow_w10_f1_500_neg2_ws2
cbow_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_500_neg5_ws5"] = cbow_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION MODELS---ASIAN")

# 700 dimensions
cbow_w3_f1_700 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, epochs = 10)
asian_cbow_models["cbow_w3_f1_700"] = cbow_w3_f1_700
cbow_w5_f1_700 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, epochs = 10)
asian_cbow_models["cbow_w5_f1_700"] = cbow_w5_f1_700
cbow_w10_f1_700 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, epochs = 10)
asian_cbow_models["cbow_w10_f1_700"] = cbow_w10_f1_700
cbow_w3_f1_700_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 0, epochs = 10)
asian_cbow_models["cbow_w3_f1_700_mc0"] = cbow_w3_f1_700_mc0
cbow_w5_f1_700_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 0, epochs = 10)
asian_cbow_models["cbow_w5_f1_700_mc0"] = cbow_w5_f1_700_mc0
cbow_w10_f1_700_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 0, epochs = 10)
asian_cbow_models["cbow_w10_f1_700_mc0"] = cbow_w10_f1_700_mc0
cbow_w10_f1_700_mc2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 0, epochs = 10)
asian_cbow_models["cbow_w10_f1_700_mc2"] = cbow_w10_f1_700_mc2
cbow_w10_f1_700_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_700_neg2"] = cbow_w10_f1_700_neg2
cbow_w10_f1_700_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_700_neg5"] = cbow_w10_f1_700_neg5
cbow_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
asian_cbow_models["cbow_w10_f1_700_ns_half_neg2"] = cbow_w10_f1_700_ns_half_neg2
cbow_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
asian_cbow_models["cbow_w10_f1_700_ns_half_neg3"] = cbow_w10_f1_700_ns_half_neg3
cbow_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
asian_cbow_models["cbow_w10_f1_700_ns_half_neg5"] = cbow_w10_f1_700_ns_half_neg5
cbow_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_700_neg2_ws2"] = cbow_w10_f1_700_neg2_ws2
cbow_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_700_neg5_ws5"] = cbow_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION MODELS---ASIAN")

# 1000 dimensions
cbow_w3_f1_1000 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
asian_cbow_models["cbow_w3_f1_1000"] = cbow_w3_f1_1000
cbow_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w2_f1_1000_ns_half_neg5"] = cbow_w2_f1_1000_ns_half_neg5
cbow_w5_f1_1000 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, epochs = 10)
asian_cbow_models["cbow_w5_f1_1000"] = cbow_w5_f1_1000
cbow_w10_f1_1000 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, epochs = 10)
asian_cbow_models["cbow_w10_f1_1000"] = cbow_w10_f1_1000
cbow_w3_f1_1000_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 10)
asian_cbow_models["cbow_w3_f1_1000_mc0"] = cbow_w3_f1_1000_mc0

cbow_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 20)
asian_cbow_models["cbow_w3_f1_1000_mc0_e20"] = cbow_w3_f1_1000_mc0_e20

cbow_w3_f1_1000_mc1 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
asian_cbow_models["cbow_w3_f1_1000_mc1"] = cbow_w3_f1_1000_mc1

cbow_w5_f1_1000_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 0, epochs = 10)
asian_cbow_models["cbow_w5_f1_1000_mc0"] = cbow_w5_f1_1000_mc0
cbow_w10_f1_1000_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 0, epochs = 10)
asian_cbow_models["cbow_w10_f1_1000_mc0"] = cbow_w10_f1_1000_mc0
cbow_w10_f1_1000_mc2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 0, epochs = 10)
asian_cbow_models["cbow_w10_f1_1000_mc2"] = cbow_w10_f1_1000_mc2
cbow_w10_f1_1000_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_1000_neg2"] = cbow_w10_f1_1000_neg2
cbow_w10_f1_1000_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_1000_neg5"] = cbow_w10_f1_1000_neg5
cbow_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
asian_cbow_models["cbow_w10_f1_1000_ns_half_neg2"] = cbow_w10_f1_1000_ns_half_neg2
cbow_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
asian_cbow_models["cbow_w10_f1_1000_ns_half_neg3"] = cbow_w10_f1_1000_ns_half_neg3
cbow_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
asian_cbow_models["cbow_w10_f1_1000_ns_half_neg5"] = cbow_w10_f1_1000_ns_half_neg5
cbow_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_1000_neg2_ws2"] = cbow_w10_f1_1000_neg2_ws2
cbow_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_cbow_models["cbow_w10_f1_1000_neg5_ws5"] = cbow_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION MODELS---ASIAN")

TRAINED ALL 100-DIMENSION MODELS---ASIAN
TRAINED ALL 300-DIMENSION MODELS---ASIAN
TRAINED ALL 500-DIMENSION MODELS---ASIAN
TRAINED ALL 700-DIMENSION MODELS---ASIAN
TRAINED ALL 1000-DIMENSION MODELS---ASIAN


In [114]:
# asian cbow save & calculate avg
synonyms = dict({"family": "household",
                 "history": "account",
                 "treatment": "intervention",
                 "social": "sociable",
                 "feeling": "belief",
                 "feeling": "opinion",
                 "clinician": "doctor",
                 "clinician": "therapist",
                })

asian_cbow_models_synonymity_average = dict()
asian_cbow_names = list(asian_cbow_models.keys())
asian_cbow_vals = list(asian_cbow_models.values())
for i in range(len(asian_cbow_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(asian_cbow_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    asian_cbow_models_synonymity_average[asian_cbow_names[i]] = average_synonimity

asian_cbow_model_chosen = max(asian_cbow_models_synonymity_average, key=asian_cbow_models_synonymity_average.get)
print(f"ASIAN CBOW MODEL: {asian_cbow_model_chosen}")
print(f"\tSYNONIMITY: {asian_cbow_models_synonymity_average[asian_cbow_model_chosen]}")
asian_cbow_models[asian_cbow_model_chosen].save(os.path.join('asian_models', asian_cbow_model_chosen))

ASIAN CBOW MODEL: cbow_w10_f1_1000_neg5
	SYNONIMITY: 0.9991270899772644


In [ ]:
# white_sg_models = {}
# # 100 dimensions
# sg_w3_f1_100 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, epochs = 10)
# white_sg_models["sg_w3_f1_100"] = sg_w3_f1_100
# sg_w5_f1_100 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, epochs = 10)
# white_sg_models["sg_w5_f1_100"] = sg_w5_f1_100
# sg_w10_f1_100 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, epochs = 10)
# white_sg_models["sg_w10_f1_100"] = sg_w10_f1_100
# sg_w3_f1_100_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 0, epochs = 10)
# white_sg_models["sg_w3_f1_100_mc0"] = sg_w3_f1_100_mc0
# sg_w5_f1_100_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 0, epochs = 10)
# white_sg_models["sg_w5_f1_100_mc0"] = sg_w5_f1_100_mc0
# sg_w10_f1_100_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 0, epochs = 10)
# white_sg_models["sg_w10_f1_100_mc0"] = sg_w10_f1_100_mc0
# sg_w10_f1_100_mc2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 0, epochs = 10)
# white_sg_models["sg_w10_f1_100_mc2"] = sg_w10_f1_100_mc2
# sg_w10_f1_100_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_100_neg2"] = sg_w10_f1_100_neg2
# sg_w10_f1_100_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_100_neg5"] = sg_w10_f1_100_neg5
# sg_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
# white_sg_models["sg_w10_f1_100_ns_half_neg2"] = sg_w10_f1_100_ns_half_neg2
# sg_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
# white_sg_models["sg_w10_f1_100_ns_half_neg3"] = sg_w10_f1_100_ns_half_neg3
# sg_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
# white_sg_models["sg_w10_f1_100_ns_half_neg5"] = sg_w10_f1_100_ns_half_neg5
# sg_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_100_neg2_ws2"] = sg_w10_f1_100_neg2_ws2
# sg_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_100_neg5_ws5"] = sg_w10_f1_100_neg5_ws5
# print("TRAINED ALL 100-DIMENSION MODELS---WHITE")

# # 300 dimensions
# sg_w3_f1_300 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, epochs = 10)
# white_sg_models["sg_w3_f1_300"] = sg_w3_f1_300
# sg_w5_f1_300 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, epochs = 10)
# white_sg_models["sg_w5_f1_300"] = sg_w5_f1_300
# sg_w10_f1_300 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, epochs = 10)
# white_sg_models["sg_w10_f1_300"] = sg_w10_f1_300
# sg_w3_f1_300_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 0, epochs = 10)
# white_sg_models["sg_w3_f1_300_mc0"] = sg_w3_f1_300_mc0
# sg_w5_f1_300_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 0, epochs = 10)
# white_sg_models["sg_w5_f1_300_mc0"] = sg_w5_f1_300_mc0
# sg_w10_f1_300_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 0, epochs = 10)
# white_sg_models["sg_w10_f1_300_mc0"] = sg_w10_f1_300_mc0
# sg_w10_f1_300_mc2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 0, epochs = 10)
# white_sg_models["sg_w10_f1_300_mc2"] = sg_w10_f1_300_mc2
# sg_w10_f1_300_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_300_neg2"] = sg_w10_f1_300_neg2
# sg_w10_f1_300_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_300_neg5"] = sg_w10_f1_300_neg5
# sg_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
# white_sg_models["sg_w10_f1_300_ns_half_neg2"] = sg_w10_f1_300_ns_half_neg2
# sg_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
# white_sg_models["sg_w10_f1_300_ns_half_neg3"] = sg_w10_f1_300_ns_half_neg3
# sg_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
# white_sg_models["sg_w10_f1_300_ns_half_neg5"] = sg_w10_f1_300_ns_half_neg5
# sg_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_300_neg2_ws2"] = sg_w10_f1_300_neg2_ws2
# sg_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_300_neg5_ws5"] = sg_w10_f1_300_neg5_ws5
# print("TRAINED ALL 300-DIMENSION MODELS---WHITE")

# # 500 dimensions
# sg_w3_f1_500 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, epochs = 10)
# white_sg_models["sg_w3_f1_500"] = sg_w3_f1_500
# sg_w5_f1_500 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, epochs = 10)
# white_sg_models["sg_w5_f1_500"] = sg_w5_f1_500
# sg_w10_f1_500 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, epochs = 10)
# white_sg_models["sg_w10_f1_500"] = sg_w10_f1_500
# sg_w3_f1_500_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 0, epochs = 10)
# white_sg_models["sg_w3_f1_500_mc0"] = sg_w3_f1_500_mc0
# sg_w5_f1_500_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 0, epochs = 10)
# white_sg_models["sg_w5_f1_500_mc0"] = sg_w5_f1_500_mc0
# sg_w10_f1_500_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 0, epochs = 10)
# white_sg_models["sg_w10_f1_500_mc0"] = sg_w10_f1_500_mc0
# sg_w10_f1_500_mc2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 0, epochs = 10)
# white_sg_models["sg_w10_f1_500_mc2"] = sg_w10_f1_500_mc2
# sg_w10_f1_500_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_500_neg2"] = sg_w10_f1_500_neg2
# sg_w10_f1_500_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_500_neg5"] = sg_w10_f1_500_neg5
# sg_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
# white_sg_models["sg_w10_f1_500_ns_half_neg2"] = sg_w10_f1_500_ns_half_neg2
# sg_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
# white_sg_models["sg_w10_f1_500_ns_half_neg3"] = sg_w10_f1_500_ns_half_neg3
# sg_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
# white_sg_models["sg_w10_f1_500_ns_half_neg5"] = sg_w10_f1_500_ns_half_neg5
# sg_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_500_neg2_ws2"] = sg_w10_f1_500_neg2_ws2
# sg_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_500_neg5_ws5"] = sg_w10_f1_500_neg5_ws5
# print("TRAINED ALL 500-DIMENSION MODELS---WHITE")

# # 700 dimensions
# sg_w3_f1_700 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, epochs = 10)
# white_sg_models["sg_w3_f1_700"] = sg_w3_f1_700
# sg_w5_f1_700 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, epochs = 10)
# white_sg_models["sg_w5_f1_700"] = sg_w5_f1_700
# sg_w10_f1_700 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, epochs = 10)
# white_sg_models["sg_w10_f1_700"] = sg_w10_f1_700
# sg_w3_f1_700_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 0, epochs = 10)
# white_sg_models["sg_w3_f1_700_mc0"] = sg_w3_f1_700_mc0
# sg_w5_f1_700_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 0, epochs = 10)
# white_sg_models["sg_w5_f1_700_mc0"] = sg_w5_f1_700_mc0
# sg_w10_f1_700_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 0, epochs = 10)
# white_sg_models["sg_w10_f1_700_mc0"] = sg_w10_f1_700_mc0
# sg_w10_f1_700_mc2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 0, epochs = 10)
# white_sg_models["sg_w10_f1_700_mc2"] = sg_w10_f1_700_mc2
# sg_w10_f1_700_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_700_neg2"] = sg_w10_f1_700_neg2
# sg_w10_f1_700_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_700_neg5"] = sg_w10_f1_700_neg5
# sg_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
# white_sg_models["sg_w10_f1_700_ns_half_neg2"] = sg_w10_f1_700_ns_half_neg2
# sg_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
# white_sg_models["sg_w10_f1_700_ns_half_neg3"] = sg_w10_f1_700_ns_half_neg3
# sg_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
# white_sg_models["sg_w10_f1_700_ns_half_neg5"] = sg_w10_f1_700_ns_half_neg5
# sg_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_700_neg2_ws2"] = sg_w10_f1_700_neg2_ws2
# sg_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_700_neg5_ws5"] = sg_w10_f1_700_neg5_ws5
# print("TRAINED ALL 700-DIMENSION MODELS---WHITE")

# # 1000 dimensions
# sg_w3_f1_1000 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
# white_sg_models["sg_w3_f1_1000"] = sg_w3_f1_1000
# sg_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w2_f1_1000_ns_half_neg5"] = sg_w2_f1_1000_ns_half_neg5
# sg_w5_f1_1000 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, epochs = 10)
# white_sg_models["sg_w5_f1_1000"] = sg_w5_f1_1000
# sg_w10_f1_1000 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, epochs = 10)
# white_sg_models["sg_w10_f1_1000"] = sg_w10_f1_1000
# sg_w3_f1_1000_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 10)
# white_sg_models["sg_w3_f1_1000_mc0"] = sg_w3_f1_1000_mc0

# sg_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 20)
# white_sg_models["sg_w3_f1_1000_mc0_e20"] = sg_w3_f1_1000_mc0_e20

# sg_w3_f1_1000_mc1 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
# white_sg_models["sg_w3_f1_1000_mc1"] = sg_w3_f1_1000_mc1

# sg_w5_f1_1000_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 0, epochs = 10)
# white_sg_models["sg_w5_f1_1000_mc0"] = sg_w5_f1_1000_mc0
# sg_w10_f1_1000_mc0 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 0, epochs = 10)
# white_sg_models["sg_w10_f1_1000_mc0"] = sg_w10_f1_1000_mc0
# sg_w10_f1_1000_mc2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 0, epochs = 10)
# white_sg_models["sg_w10_f1_1000_mc2"] = sg_w10_f1_1000_mc2
# sg_w10_f1_1000_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_1000_neg2"] = sg_w10_f1_1000_neg2
# sg_w10_f1_1000_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_1000_neg5"] = sg_w10_f1_1000_neg5
# sg_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
# white_sg_models["sg_w10_f1_1000_ns_half_neg2"] = sg_w10_f1_1000_ns_half_neg2
# sg_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
# white_sg_models["sg_w10_f1_1000_ns_half_neg3"] = sg_w10_f1_1000_ns_half_neg3
# sg_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
# white_sg_models["sg_w10_f1_1000_ns_half_neg5"] = sg_w10_f1_1000_ns_half_neg5
# sg_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_1000_neg2_ws2"] = sg_w10_f1_1000_neg2_ws2
# sg_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(white_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
# white_sg_models["sg_w10_f1_1000_neg5_ws5"] = sg_w10_f1_1000_neg5_ws5
# print("TRAINED ALL 1000-DIMENSION MODELS---WHITE")

In [ ]:
# # white sg save & calculate avg
# synonyms = dict({"assessment": "appraisal",
#                  "medication": "medicine",
#                  "appointment": "engagement",
#                  "road": "route",
#                  "family": "household",
#                  "history": "account",
#                  "mood": "temper",
#                  "information": "info",
#                  "treatment": "intervention",
#                  "social": "sociable",
#                  "anxious": "nervous",
#                  "confidential": "private",
#                  "feeling": "belief",
#                  "feeling": "opinion"
#                 })

# white_sg_models_synonymity_average = {}
# white_sg_names = list(white_sg_models.keys())
# white_sg_vals = list(white_sg_models.values())
# for i in range(len(white_sg_names)):
#     average_synonimity = 0
#     synonymities = list()
#     for s1 in synonyms:
#         synonymities.append(white_sg_vals[i].wv.similarity(s1, synonyms[s1]))
#     average_synonimity = mean(synonymities)
#     white_sg_models_synonymity_average[white_sg_names[i]] = average_synonimity

# white_sg_model_chosen = max(white_sg_models_synonymity_average, key=white_sg_models_synonymity_average.get)
# print(f"WHITE SG MODEL: {white_sg_model_chosen}")
# print(f"\tSYNONIMITY: {white_sg_models_synonymity_average[white_sg_model_chosen]}")
# white_sg_models[white_sg_model_chosen].save(os.path.join('white_models', white_sg_model_chosen))

In [ ]:
# black_sg_models = {}
# # 100 dimensions
# sg_w3_f1_100 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, epochs = 10)
# black_sg_models["sg_w3_f1_100"] = sg_w3_f1_100
# sg_w5_f1_100 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, epochs = 10)
# black_sg_models["sg_w5_f1_100"] = sg_w5_f1_100
# sg_w10_f1_100 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, epochs = 10)
# black_sg_models["sg_w10_f1_100"] = sg_w10_f1_100
# sg_w3_f1_100_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 1, epochs = 10)
# black_sg_models["sg_w3_f1_100_mc0"] = sg_w3_f1_100_mc0
# sg_w5_f1_100_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 1, epochs = 10)
# black_sg_models["sg_w5_f1_100_mc0"] = sg_w5_f1_100_mc0
# sg_w10_f1_100_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 1, epochs = 10)
# black_sg_models["sg_w10_f1_100_mc0"] = sg_w10_f1_100_mc0
# sg_w10_f1_100_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_100_neg2"] = sg_w10_f1_100_neg2
# sg_w10_f1_100_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_100_neg5"] = sg_w10_f1_100_neg5
# sg_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
# black_sg_models["sg_w10_f1_100_ns_half_neg2"] = sg_w10_f1_100_ns_half_neg2
# sg_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
# black_sg_models["sg_w10_f1_100_ns_half_neg5"] = sg_w10_f1_100_ns_half_neg5
# sg_w2_f1_100_neg2_ws2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w2_f1_100_neg2_ws2"] = sg_w2_f1_100_neg2_ws2
# sg_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_100_neg5_ws5"] = sg_w10_f1_100_neg5_ws5
# print("TRAINED ALL 100-DIMENSION MODELS---BLACK")

# # 300 dimensions
# sg_w3_f1_300 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, epochs = 10)
# black_sg_models["sg_w3_f1_300"] = sg_w3_f1_300
# sg_w5_f1_300 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, epochs = 10)
# black_sg_models["sg_w5_f1_300"] = sg_w5_f1_300
# sg_w10_f1_300 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, epochs = 10)
# black_sg_models["sg_w10_f1_300"] = sg_w10_f1_300
# sg_w3_f1_300_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 1, epochs = 10)
# black_sg_models["sg_w3_f1_300_mc0"] = sg_w3_f1_300_mc0
# sg_w5_f1_300_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 1, epochs = 10)
# black_sg_models["sg_w5_f1_300_mc0"] = sg_w5_f1_300_mc0
# sg_w10_f1_300_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 1, epochs = 10)
# black_sg_models["sg_w10_f1_300_mc0"] = sg_w10_f1_300_mc0
# sg_w10_f1_300_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_300_neg2"] = sg_w10_f1_300_neg2
# sg_w10_f1_300_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_300_neg5"] = sg_w10_f1_300_neg5
# sg_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
# black_sg_models["sg_w10_f1_300_ns_half_neg2"] = sg_w10_f1_300_ns_half_neg2
# sg_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
# black_sg_models["sg_w10_f1_300_ns_half_neg5"] = sg_w10_f1_300_ns_half_neg5
# sg_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_300_neg2_ws2"] = sg_w10_f1_300_neg2_ws2
# sg_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_300_neg5_ws5"] = sg_w10_f1_300_neg5_ws5
# print("TRAINED ALL 300-DIMENSION MODELS---BLACK")

# # 500 dimensions
# sg_w3_f1_500 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, epochs = 10)
# black_sg_models["sg_w3_f1_500"] = sg_w3_f1_500
# sg_w5_f1_500 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, epochs = 10)
# black_sg_models["sg_w5_f1_500"] = sg_w5_f1_500
# sg_w10_f1_500 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, epochs = 10)
# black_sg_models["sg_w10_f1_500"] = sg_w10_f1_500
# sg_w3_f1_500_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 1, epochs = 10)
# black_sg_models["sg_w3_f1_500_mc0"] = sg_w3_f1_500_mc0
# sg_w5_f1_500_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 1, epochs = 10)
# black_sg_models["sg_w5_f1_500_mc0"] = sg_w5_f1_500_mc0
# sg_w10_f1_500_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 1, epochs = 10)
# black_sg_models["sg_w10_f1_500_mc0"] = sg_w10_f1_500_mc0
# sg_w10_f1_500_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_500_neg2"] = sg_w10_f1_500_neg2
# sg_w10_f1_500_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_500_neg5"] = sg_w10_f1_500_neg5
# sg_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
# black_sg_models["sg_w10_f1_500_ns_half_neg2"] = sg_w10_f1_500_ns_half_neg2
# sg_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
# black_sg_models["sg_w10_f1_500_ns_half_neg5"] = sg_w10_f1_500_ns_half_neg5
# sg_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_500_neg2_ws2"] = sg_w10_f1_500_neg2_ws2
# sg_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_500_neg5_ws5"] = sg_w10_f1_500_neg5_ws5
# print("TRAINED ALL 500-DIMENSION MODELS---BLACK")

# # 700 dimensions
# sg_w3_f1_700 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, epochs = 10)
# black_sg_models["sg_w3_f1_700"] = sg_w3_f1_700
# sg_w5_f1_700 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, epochs = 10)
# black_sg_models["sg_w5_f1_700"] = sg_w5_f1_700
# sg_w10_f1_700 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, epochs = 10)
# black_sg_models["sg_w10_f1_700"] = sg_w10_f1_700
# sg_w3_f1_700_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 1, epochs = 10)
# black_sg_models["sg_w3_f1_700_mc0"] = sg_w3_f1_700_mc0
# sg_w5_f1_700_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 1, epochs = 10)
# black_sg_models["sg_w5_f1_700_mc0"] = sg_w5_f1_700_mc0
# sg_w10_f1_700_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 1, epochs = 10)
# black_sg_models["sg_w10_f1_700_mc0"] = sg_w10_f1_700_mc0
# sg_w10_f1_700_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_700_neg2"] = sg_w10_f1_700_neg2
# sg_w10_f1_700_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_700_neg5"] = sg_w10_f1_700_neg5
# sg_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
# black_sg_models["sg_w10_f1_700_ns_half_neg2"] = sg_w10_f1_700_ns_half_neg2
# sg_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
# black_sg_models["sg_w10_f1_700_ns_half_neg5"] = sg_w10_f1_700_ns_half_neg5
# sg_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_700_neg2_ws2"] = sg_w10_f1_700_neg2_ws2
# sg_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_700_neg5_ws5"] = sg_w10_f1_700_neg5_ws5
# print("TRAINED ALL 700-DIMENSION MODELS---BLACK")

# # 1000 dimensions
# sg_w3_f1_1000 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
# black_sg_models["sg_w3_f1_1000"] = sg_w3_f1_1000
# sg_w5_f1_1000 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, epochs = 10)
# black_sg_models["sg_w5_f1_1000"] = sg_w5_f1_1000
# sg_w10_f1_1000 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, epochs = 10)
# black_sg_models["sg_w10_f1_1000"] = sg_w10_f1_1000
# sg_w3_f1_1000_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 10)
# black_sg_models["sg_w3_f1_1000_mc0"] = sg_w3_f1_1000_mc0
# sg_w5_f1_1000_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 1, epochs = 10)
# black_sg_models["sg_w5_f1_1000_mc0"] = sg_w5_f1_1000_mc0
# sg_w10_f1_1000_mc0 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 1, epochs = 10)
# black_sg_models["sg_w10_f1_1000_mc0"] = sg_w10_f1_1000_mc0
# sg_w10_f1_1000_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_1000_neg2"] = sg_w10_f1_1000_neg2
# sg_w10_f1_1000_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_1000_neg5"] = sg_w10_f1_1000_neg5
# sg_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
# black_sg_models["sg_w10_f1_1000_ns_half_neg2"] = sg_w10_f1_1000_ns_half_neg2
# sg_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
# black_sg_models["sg_w10_f1_1000_ns_half_neg5"] = sg_w10_f1_1000_ns_half_neg5
# sg_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_1000_neg2_ws2"] = sg_w10_f1_1000_neg2_ws2
# sg_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(black_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# black_sg_models["sg_w10_f1_1000_neg5_ws5"] = sg_w10_f1_1000_neg5_ws5
# print("TRAINED ALL 1000-DIMENSION MODELS---BLACK")

In [ ]:
# # black sg save & calculate avg
# synonyms = dict({"medication": "medicine",
#                  "appointment": "engagement",
#                  "road": "route",
#                  "family": "household",
#                  "history": "account",
#                  "mood": "temper",
#                  "information": "info",
#                  "treatment": "intervention",
#                  "anxious": "nervous",
#                  "confidential": "private",
#                  "feeling": "belief",
#                  "feeling": "opinion"
#                 })

# black_sg_models_synonymity_average = {}
# black_sg_names = list(black_sg_models.keys())
# black_sg_vals = list(black_sg_models.values())
# for i in range(len(black_sg_names)):
#     average_synonimity = 0
#     synonymities = list()
#     for s1 in synonyms:
#         synonymities.append(black_sg_vals[i].wv.similarity(s1, synonyms[s1]))
#     average_synonimity = mean(synonymities)
#     black_sg_models_synonymity_average[black_sg_names[i]] = average_synonimity

# black_sg_model_chosen = max(black_sg_models_synonymity_average, key=black_sg_models_synonymity_average.get)
# print(f"BLACK SG MODEL: {black_sg_model_chosen}")
# print(f"\tSYNONIMITY: {black_sg_models_synonymity_average[black_sg_model_chosen]}")
# black_sg_vals[black_sg_model_chosen].save(os.path.join('black_models', black_sg_model_chosen))

In [ ]:
# mixed_sg_models = {}
# # 100 dimensions
# sg_w3_f1_100 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, epochs = 10)
# mixed_sg_models["sg_w3_f1_100"] = sg_w3_f1_100
# sg_w5_f1_100 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, epochs = 10)
# mixed_sg_models["sg_w5_f1_100"] = sg_w5_f1_100
# sg_w10_f1_100 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, epochs = 10)
# mixed_sg_models["sg_w10_f1_100"] = sg_w10_f1_100
# sg_w3_f1_100_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 1, epochs = 10)
# mixed_sg_models["sg_w3_f1_100_mc0"] = sg_w3_f1_100_mc0
# sg_w5_f1_100_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 1, epochs = 10)
# mixed_sg_models["sg_w5_f1_100_mc0"] = sg_w5_f1_100_mc0
# sg_w10_f1_100_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 1, epochs = 10)
# mixed_sg_models["sg_w10_f1_100_mc0"] = sg_w10_f1_100_mc0
# sg_w10_f1_100_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_100_neg2"] = sg_w10_f1_100_neg2
# sg_w10_f1_100_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_100_neg5"] = sg_w10_f1_100_neg5
# sg_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
# mixed_sg_models["sg_w10_f1_100_ns_half_neg2"] = sg_w10_f1_100_ns_half_neg2
# sg_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
# mixed_sg_models["sg_w10_f1_100_ns_half_neg5"] = sg_w10_f1_100_ns_half_neg5
# sg_w2_f1_100_neg2_ws2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w2_f1_100_neg2_ws2"] = sg_w2_f1_100_neg2_ws2
# sg_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_100_neg5_ws5"] = sg_w10_f1_100_neg5_ws5
# print("TRAINED ALL 100-DIMENSION MODELS---MIXED")

# # 300 dimensions
# sg_w3_f1_300 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, epochs = 10)
# mixed_sg_models["sg_w3_f1_300"] = sg_w3_f1_300
# sg_w5_f1_300 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, epochs = 10)
# mixed_sg_models["sg_w5_f1_300"] = sg_w5_f1_300
# sg_w10_f1_300 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, epochs = 10)
# mixed_sg_models["sg_w10_f1_300"] = sg_w10_f1_300
# sg_w3_f1_300_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 1, epochs = 10)
# mixed_sg_models["sg_w3_f1_300_mc0"] = sg_w3_f1_300_mc0
# sg_w5_f1_300_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 1, epochs = 10)
# mixed_sg_models["sg_w5_f1_300_mc0"] = sg_w5_f1_300_mc0
# sg_w10_f1_300_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 1, epochs = 10)
# mixed_sg_models["sg_w10_f1_300_mc0"] = sg_w10_f1_300_mc0
# sg_w10_f1_300_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_300_neg2"] = sg_w10_f1_300_neg2
# sg_w10_f1_300_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_300_neg5"] = sg_w10_f1_300_neg5
# sg_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
# mixed_sg_models["sg_w10_f1_300_ns_half_neg2"] = sg_w10_f1_300_ns_half_neg2
# sg_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
# mixed_sg_models["sg_w10_f1_300_ns_half_neg5"] = sg_w10_f1_300_ns_half_neg5
# sg_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_300_neg2_ws2"] = sg_w10_f1_300_neg2_ws2
# sg_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_300_neg5_ws5"] = sg_w10_f1_300_neg5_ws5
# print("TRAINED ALL 300-DIMENSION MODELS---MIXED")

# # 500 dimensions
# sg_w3_f1_500 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, epochs = 10)
# mixed_sg_models["sg_w3_f1_500"] = sg_w3_f1_500
# sg_w5_f1_500 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, epochs = 10)
# mixed_sg_models["sg_w5_f1_500"] = sg_w5_f1_500
# sg_w10_f1_500 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, epochs = 10)
# mixed_sg_models["sg_w10_f1_500"] = sg_w10_f1_500
# sg_w3_f1_500_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 1, epochs = 10)
# mixed_sg_models["sg_w3_f1_500_mc0"] = sg_w3_f1_500_mc0
# sg_w5_f1_500_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 1, epochs = 10)
# mixed_sg_models["sg_w5_f1_500_mc0"] = sg_w5_f1_500_mc0
# sg_w10_f1_500_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 1, epochs = 10)
# mixed_sg_models["sg_w10_f1_500_mc0"] = sg_w10_f1_500_mc0
# sg_w10_f1_500_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_500_neg2"] = sg_w10_f1_500_neg2
# sg_w10_f1_500_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_500_neg5"] = sg_w10_f1_500_neg5
# sg_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
# mixed_sg_models["sg_w10_f1_500_ns_half_neg2"] = sg_w10_f1_500_ns_half_neg2
# sg_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
# mixed_sg_models["sg_w10_f1_500_ns_half_neg5"] = sg_w10_f1_500_ns_half_neg5
# sg_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_500_neg2_ws2"] = sg_w10_f1_500_neg2_ws2
# sg_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_500_neg5_ws5"] = sg_w10_f1_500_neg5_ws5
# print("TRAINED ALL 500-DIMENSION MODELS---MIXED")

# # 700 dimensions
# sg_w3_f1_700 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, epochs = 10)
# mixed_sg_models["sg_w3_f1_700"] = sg_w3_f1_700
# sg_w5_f1_700 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, epochs = 10)
# mixed_sg_models["sg_w5_f1_700"] = sg_w5_f1_700
# sg_w10_f1_700 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, epochs = 10)
# mixed_sg_models["sg_w10_f1_700"] = sg_w10_f1_700
# sg_w3_f1_700_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 1, epochs = 10)
# mixed_sg_models["sg_w3_f1_700_mc0"] = sg_w3_f1_700_mc0
# sg_w5_f1_700_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 1, epochs = 10)
# mixed_sg_models["sg_w5_f1_700_mc0"] = sg_w5_f1_700_mc0
# sg_w10_f1_700_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 1, epochs = 10)
# mixed_sg_models["sg_w10_f1_700_mc0"] = sg_w10_f1_700_mc0
# sg_w10_f1_700_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_700_neg2"] = sg_w10_f1_700_neg2
# sg_w10_f1_700_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_700_neg5"] = sg_w10_f1_700_neg5
# sg_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
# mixed_sg_models["sg_w10_f1_700_ns_half_neg2"] = sg_w10_f1_700_ns_half_neg2
# sg_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
# mixed_sg_models["sg_w10_f1_700_ns_half_neg5"] = sg_w10_f1_700_ns_half_neg5
# sg_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_700_neg2_ws2"] = sg_w10_f1_700_neg2_ws2
# sg_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_700_neg5_ws5"] = sg_w10_f1_700_neg5_ws5
# print("TRAINED ALL 700-DIMENSION MODELS---MIXED")

# # 1000 dimensions
# sg_w3_f1_1000 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
# mixed_sg_models["sg_w3_f1_1000"] = sg_w3_f1_1000
# sg_w5_f1_1000 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, epochs = 10)
# mixed_sg_models["sg_w5_f1_1000"] = sg_w5_f1_1000
# sg_w10_f1_1000 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, epochs = 10)
# mixed_sg_models["sg_w10_f1_1000"] = sg_w10_f1_1000
# sg_w3_f1_1000_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 10)
# mixed_sg_models["sg_w3_f1_1000_mc0"] = sg_w3_f1_1000_mc0
# sg_w5_f1_1000_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 1, epochs = 10)
# mixed_sg_models["sg_w5_f1_1000_mc0"] = sg_w5_f1_1000_mc0
# sg_w10_f1_1000_mc0 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 1, epochs = 10)
# mixed_sg_models["sg_w10_f1_1000_mc0"] = sg_w10_f1_1000_mc0
# sg_w10_f1_1000_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_1000_neg2"] = sg_w10_f1_1000_neg2
# sg_w10_f1_1000_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_1000_neg5"] = sg_w10_f1_1000_neg5
# sg_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
# mixed_sg_models["sg_w10_f1_1000_ns_half_neg2"] = sg_w10_f1_1000_ns_half_neg2
# sg_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
# mixed_sg_models["sg_w10_f1_1000_ns_half_neg5"] = sg_w10_f1_1000_ns_half_neg5
# sg_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_1000_neg2_ws2"] = sg_w10_f1_1000_neg2_ws2
# sg_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(mixed_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# mixed_sg_models["sg_w10_f1_1000_neg5_ws5"] = sg_w10_f1_1000_neg5_ws5
# print("TRAINED ALL 1000-DIMENSION MODELS---MIXED")
# print("FINISHED TRAINING SKIP-GRAM WORD2VEC MODELS")

In [ ]:
# # mixed sg save & calculate avg
# synonyms = dict({"medication": "medicine",
#                  "appointment": "engagement",
#                  "family": "household",
#                  "history": "account",
#                  "information": "info",
#                  "treatment": "intervention"
#                 })

# mixed_sg_models_synonymity_average = dict()
# mixed_sg_names = list(mixed_sg_models.keys())
# mixed_sg_vals = list(mixed_sg_models.values())
# for i in range(len(mixed_sg_names)):
#     average_synonimity = 0
#     synonymities = list()
#     for s1 in synonyms:
#         synonymities.append(mixed_sg_vals[i].wv.similarity(s1, synonyms[s1]))
#     average_synonimity = mean(synonymities)
#     mixed_sg_models_synonymity_average[mixed_sg_names[i]] = average_synonimity

# mixed_sg_model_chosen = max(mixed_sg_models_synonymity_average, key=mixed_sg_models_synonymity_average.get)
# print(f"MIXED SG MODEL: {mixed_sg_model_chosen}")
# print(f"\tSYNONIMITY: {mixed_sg_models_synonymity_average[mixed_sg_model_chosen]}")
# mixed_sg_vals[mixed_sg_model_chosen].save(os.path.join('mixed_models', mixed_sg_model_chosen))

In [ ]:
# asian_sg_models = {}
# # 100 dimensions
# sg_w3_f1_100 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, epochs = 10)
# asian_sg_models["sg_w3_f1_100"] = sg_w3_f1_100
# sg_w5_f1_100 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, epochs = 10)
# asian_sg_models["sg_w5_f1_100"] = sg_w5_f1_100
# sg_w10_f1_100 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, epochs = 10)
# asian_sg_models["sg_w10_f1_100"] = sg_w10_f1_100
# sg_w3_f1_100_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 1, epochs = 10)
# asian_sg_models["sg_w3_f1_100_mc0"] = sg_w3_f1_100_mc0
# sg_w5_f1_100_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 1, epochs = 10)
# asian_sg_models["sg_w5_f1_100_mc0"] = sg_w5_f1_100_mc0
# sg_w10_f1_100_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 1, epochs = 10)
# asian_sg_models["sg_w10_f1_100_mc0"] = sg_w10_f1_100_mc0
# sg_w10_f1_100_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_100_neg2"] = sg_w10_f1_100_neg2
# sg_w10_f1_100_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_100_neg5"] = sg_w10_f1_100_neg5
# sg_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
# asian_sg_models["sg_w10_f1_100_ns_half_neg2"] = sg_w10_f1_100_ns_half_neg2
# sg_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
# asian_sg_models["sg_w10_f1_100_ns_half_neg5"] = sg_w10_f1_100_ns_half_neg5
# sg_w2_f1_100_neg2_ws2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w2_f1_100_neg2_ws2"] = sg_w2_f1_100_neg2_ws2
# sg_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_100_neg5_ws5"] = sg_w10_f1_100_neg5_ws5
# print("TRAINED ALL 100-DIMENSION MODELS---ASIAN")

# # 300 dimensions
# sg_w3_f1_300 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, epochs = 10)
# asian_sg_models["sg_w3_f1_300"] = sg_w3_f1_300
# sg_w5_f1_300 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, epochs = 10)
# asian_sg_models["sg_w5_f1_300"] = sg_w5_f1_300
# sg_w10_f1_300 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, epochs = 10)
# asian_sg_models["sg_w10_f1_300"] = sg_w10_f1_300
# sg_w3_f1_300_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 1, epochs = 10)
# asian_sg_models["sg_w3_f1_300_mc0"] = sg_w3_f1_300_mc0
# sg_w5_f1_300_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 1, epochs = 10)
# asian_sg_models["sg_w5_f1_300_mc0"] = sg_w5_f1_300_mc0
# sg_w10_f1_300_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 1, epochs = 10)
# asian_sg_models["sg_w10_f1_300_mc0"] = sg_w10_f1_300_mc0
# sg_w10_f1_300_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_300_neg2"] = sg_w10_f1_300_neg2
# sg_w10_f1_300_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_300_neg5"] = sg_w10_f1_300_neg5
# sg_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
# asian_sg_models["sg_w10_f1_300_ns_half_neg2"] = sg_w10_f1_300_ns_half_neg2
# sg_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
# asian_sg_models["sg_w10_f1_300_ns_half_neg5"] = sg_w10_f1_300_ns_half_neg5
# sg_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_300_neg2_ws2"] = sg_w10_f1_300_neg2_ws2
# sg_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_300_neg5_ws5"] = sg_w10_f1_300_neg5_ws5
# print("TRAINED ALL 300-DIMENSION MODELS---ASIAN")

# # 500 dimensions
# sg_w3_f1_500 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, epochs = 10)
# asian_sg_models["sg_w3_f1_500"] = sg_w3_f1_500
# sg_w5_f1_500 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, epochs = 10)
# asian_sg_models["sg_w5_f1_500"] = sg_w5_f1_500
# sg_w10_f1_500 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, epochs = 10)
# asian_sg_models["sg_w10_f1_500"] = sg_w10_f1_500
# sg_w3_f1_500_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 1, epochs = 10)
# asian_sg_models["sg_w3_f1_500_mc0"] = sg_w3_f1_500_mc0
# sg_w5_f1_500_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 1, epochs = 10)
# asian_sg_models["sg_w5_f1_500_mc0"] = sg_w5_f1_500_mc0
# sg_w10_f1_500_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 1, epochs = 10)
# asian_sg_models["sg_w10_f1_500_mc0"] = sg_w10_f1_500_mc0
# sg_w10_f1_500_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_500_neg2"] = sg_w10_f1_500_neg2
# sg_w10_f1_500_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_500_neg5"] = sg_w10_f1_500_neg5
# sg_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
# asian_sg_models["sg_w10_f1_500_ns_half_neg2"] = sg_w10_f1_500_ns_half_neg2
# sg_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
# asian_sg_models["sg_w10_f1_500_ns_half_neg5"] = sg_w10_f1_500_ns_half_neg5
# sg_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_500_neg2_ws2"] = sg_w10_f1_500_neg2_ws2
# sg_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_500_neg5_ws5"] = sg_w10_f1_500_neg5_ws5
# print("TRAINED ALL 500-DIMENSION MODELS---ASIAN")

# # 700 dimensions
# sg_w3_f1_700 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, epochs = 10)
# asian_sg_models["sg_w3_f1_700"] = sg_w3_f1_700
# sg_w5_f1_700 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, epochs = 10)
# asian_sg_models["sg_w5_f1_700"] = sg_w5_f1_700
# sg_w10_f1_700 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, epochs = 10)
# asian_sg_models["sg_w10_f1_700"] = sg_w10_f1_700
# sg_w3_f1_700_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 1, epochs = 10)
# asian_sg_models["sg_w3_f1_700_mc0"] = sg_w3_f1_700_mc0
# sg_w5_f1_700_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 1, epochs = 10)
# asian_sg_models["sg_w5_f1_700_mc0"] = sg_w5_f1_700_mc0
# sg_w10_f1_700_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 1, epochs = 10)
# asian_sg_models["sg_w10_f1_700_mc0"] = sg_w10_f1_700_mc0
# sg_w10_f1_700_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_700_neg2"] = sg_w10_f1_700_neg2
# sg_w10_f1_700_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_700_neg5"] = sg_w10_f1_700_neg5
# sg_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
# asian_sg_models["sg_w10_f1_700_ns_half_neg2"] = sg_w10_f1_700_ns_half_neg2
# sg_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
# asian_sg_models["sg_w10_f1_700_ns_half_neg5"] = sg_w10_f1_700_ns_half_neg5
# sg_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_700_neg2_ws2"] = sg_w10_f1_700_neg2_ws2
# sg_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_700_neg5_ws5"] = sg_w10_f1_700_neg5_ws5
# print("TRAINED ALL 700-DIMENSION MODELS---ASIAN")

# # 1000 dimensions
# sg_w3_f1_1000 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
# asian_sg_models["sg_w3_f1_1000"] = sg_w3_f1_1000
# sg_w5_f1_1000 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, epochs = 10)
# asian_sg_models["sg_w5_f1_1000"] = sg_w5_f1_1000
# sg_w10_f1_1000 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, epochs = 10)
# asian_sg_models["sg_w10_f1_1000"] = sg_w10_f1_1000
# sg_w3_f1_1000_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 10)
# asian_sg_models["sg_w3_f1_1000_mc0"] = sg_w3_f1_1000_mc0
# sg_w5_f1_1000_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 1, epochs = 10)
# asian_sg_models["sg_w5_f1_1000_mc0"] = sg_w5_f1_1000_mc0
# sg_w10_f1_1000_mc0 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 1, epochs = 10)
# asian_sg_models["sg_w10_f1_1000_mc0"] = sg_w10_f1_1000_mc0
# sg_w10_f1_1000_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_1000_neg2"] = sg_w10_f1_1000_neg2
# sg_w10_f1_1000_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_1000_neg5"] = sg_w10_f1_1000_neg5
# sg_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
# asian_sg_models["sg_w10_f1_1000_ns_half_neg2"] = sg_w10_f1_1000_ns_half_neg2
# sg_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
# asian_sg_models["sg_w10_f1_1000_ns_half_neg5"] = sg_w10_f1_1000_ns_half_neg5
# sg_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_1000_neg2_ws2"] = sg_w10_f1_1000_neg2_ws2
# sg_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(asian_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
# asian_sg_models["sg_w10_f1_1000_neg5_ws5"] = sg_w10_f1_1000_neg5_ws5
# print("TRAINED ALL 1000-DIMENSION MODELS---ASIAN")
# print("FINISHED TRAINING SKIP-GRAM WORD2VEC MODELS")

In [ ]:
# # asian sg save & calculate avg
# synonyms = dict({"medication": "medicine",
#                  "appointment": "engagement",
#                  "road": "route",
#                  "family": "household",
#                  "history": "account",
#                  "information": "info",
#                  "treatment": "intervention",
#                  "social": "sociable",
#                  "confidential": "private",
#                  "feeling": "belief",
#                  "feeling": "opinion"
#                 })

# asian_sg_models_synonymity_average = dict()
# asian_sg_names = list(asian_sg_models.keys())
# asian_sg_vals = list(asian_sg_models.values())
# for i in range(len(asian_sg_names)):
#     average_synonimity = 0
#     synonymities = list()
#     for s1 in synonyms:
#         synonymities.append(asian_sg_vals[i].wv.similarity(s1, synonyms[s1]))
#     average_synonimity = mean(synonymities)
#     asian_sg_models_synonymity_average[asian_sg_names[i]] = average_synonimity

# asian_sg_model_chosen = max(asian_sg_models_synonymity_average, key=asian_sg_models_synonymity_average.get)
# print(f"ASIAN SG MODEL: {asian_sg_model_chosen}")
# print(f"\tSYNONIMITY: {asian_sg_models_synonymity_average[asian_sg_model_chosen]}")
# asian_sg_vals[asian_sg_model_chosen].save(os.path.join('asian_models', asian_sg_model_chosen))

In [38]:
# GROUP 1 VOCAB: Symptoms
symptoms = ['delusional', 'manic', 'suicidal', 'delusions', 'mania', 'suicide', 'disordered', 'eating', 'insomnia', 'addiction', 'addicted', 'weight']

# GROUP 2 VOCAB: Conditions
conditions = ['pregnant', 'pregnancy', 'schizophrenia', 'depression', 'anxiety', 'schizophrenic', 'depressed', 'anxious', 'bipolar_disorder', 'bipolar', 'autism', 'autistic', 'handicapped', 'handicap', 'learning', 'diabetes', 'obese', 'obesity', 'alcoholism', 'alcohol']

# GROUP 3 VOCAB: Interpersonal
interpersonal = ['family', 'relationship', 'mother', 'father', 'husband', 'wife', 'son', 'daughter', 'child', 'parent', 'parents', 'friends', 'support', 'marriage', 'partner', 'childcare']

# GROUP 4 VOCAB: Behaviour
behaviour = ['mood', 'behaviour', 'violent', 'agitated', 'moody', 'withdrawn', 'aggressive', 'submissive', 'quiet', 'speech', 'listen', 'sleep', 'irritable']

# GROUP 5 VOCAB: Clinical
clinical = ['appointment', 'assessment', 'cancel', 'reschedule', 'treatment', 'schedule', 'intervention', 'medication', 'medicate', 'medicine', 'doctor', 'therapist', 'clinician']

# GROUP 6 VOCAB: Stereotypes
stereotypes = ['articulate', 'english', 'native', 'foreign', 'immigrant', 'immigration', 'visa', 'ethnic', 'cultural', 'culture']

# GROUP 7 VOCAB: Subjective
subjective = ['stubborn', 'frustrating', 'defiant', 'obstinate', 'unwilling', 'risk', 'personality', 'opinion', 'believe', 'seem', 'claim']

In [116]:
print("WHITE CBOW TOP VOCAB: ") # white_cbow_model_chosen
print_vocab(white_cbow_models[white_cbow_model_chosen], top_n = 50)
# print("\nWHITE SG TOP VOCAB: ")
# print_vocab(white_sg_models[white_sg_model_chosen], top_n = 50)

print("\nBLACK CBOW TOP VOCAB: ")
print_vocab(black_cbow_models[black_cbow_model_chosen], top_n = 50)
# print("\nBLACK SG TOP VOCAB: ")
# print_vocab(black_sg_models[black_sg_model_chosen], top_n = 50)

print("\nMIXED CBOW TOP VOCAB: ")
print_vocab(mixed_cbow_models[mixed_cbow_model_chosen], top_n = 50)
# print("\nMIXED SG TOP VOCAB: ")
# print_vocab(mixed_sg_models[mixed_sg_model_chosen], top_n = 50)

print("\nASIAN CBOW TOP VOCAB: ")
print_vocab(asian_cbow_models[asian_cbow_model_chosen], top_n = 50)
# print("\nASIAN SG TOP VOCAB: ")
# print_vocab(asian_sg_models[asian_sg_model_chosen], top_n = 50)

WHITE CBOW TOP VOCAB: 
WORD #0/12958 IS: Dr
WORD #1/12958 IS: please
WORD #2/12958 IS: appointment
WORD #3/12958 IS: feel
WORD #4/12958 IS: service
WORD #5/12958 IS: time
WORD #6/12958 IS: contact
WORD #7/12958 IS: would
WORD #8/12958 IS: report
WORD #9/12958 IS: Team
WORD #10/12958 IS: Road
WORD #11/12958 IS: also
WORD #12/12958 IS: see
WORD #13/12958 IS: support
WORD #14/12958 IS: year
WORD #15/12958 IS: assessment
WORD #16/12958 IS: mood
WORD #17/12958 IS: need
WORD #18/12958 IS: mental
WORD #19/12958 IS: attend
WORD #20/12958 IS: health
WORD #21/12958 IS: Mr
WORD #22/12958 IS: work
WORD #23/12958 IS: GP
WORD #24/12958 IS: take
WORD #25/12958 IS: risk
WORD #26/12958 IS: medication
WORD #27/12958 IS: child
WORD #28/12958 IS: referral
WORD #29/12958 IS: Hospital
WORD #30/12958 IS: sincerely
WORD #31/12958 IS: care
WORD #32/12958 IS: history
WORD #33/12958 IS: number
WORD #34/12958 IS: well
WORD #35/12958 IS: self
WORD #36/12958 IS: Service
WORD #37/12958 IS: plan
WORD #38/12958 IS: re

In [124]:
word_embeddings = {}
for word in symptoms:
    print(f"\nWORD: {word.upper()}")
    word_file = open(f"results/symptoms_25/{word}.txt", "a")
    try:
        key = f"{word}_white_cbow"
        word_embeddings[key] = []
        word_file.write("---------WHITE (CBOW)---------\n")
        print(white_cbow_models[white_cbow_model_chosen].wv.similar_by_word(word, 25))
        for entry in white_cbow_models[white_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_black_cbow"
        word_embeddings[key] = []
        word_file.write("\n---------BLACK (CBOW)---------\n")
        for entry in black_cbow_models[black_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_mixed_cbow"
        word_embeddings[key] = []
        word_file.write("\n---------MIXED (CBOW)---------\n")
        for entry in mixed_cbow_models[mixed_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_asian_cbow"
        word_embeddings[key] = []
        word_file.write("---------ASIAN (CBOW)---------\n")
        for entry in asian_cbow_models[asian_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_white_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------WHITE (SG)---------\n")
        # for entry in black_cbow_models[black_cbow_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_black_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------BLACK (SG)---------\n")
        # for entry in black_sg_models[black_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_mixed_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------MIXED (SG)---------\n")
        # for entry in mixed_sg_models[mixed_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_asian_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------ASIAN (SG)---------\n")
        # for entry in asian_sg_models[asian_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
        # word_file.close()
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

for word in conditions:
    print(f"\nWORD: {word.upper()}")
    word_file = open(f"results/conditions_25/{word}.txt", "a")
    try:
        key = f"{word}_white_cbow"
        word_embeddings[key] = []
        word_file.write("---------WHITE (CBOW)---------\n")
        for entry in white_cbow_models[white_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_black_cbow"
        word_embeddings[key] = []
        word_file.write("---------BLACK (CBOW)---------\n")
        for entry in black_cbow_models[black_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_mixed_cbow"
        word_embeddings[key] = []
        word_file.write("\n---------MIXED (CBOW)---------\n")
        for entry in mixed_cbow_models[mixed_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_asian_cbow"
        word_embeddings[key] = []
        word_file.write("---------ASIAN (CBOW)---------\n")
        for entry in asian_cbow_models[asian_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_white_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------WHITE (SG)---------\n")
        # for entry in black_cbow_models[black_cbow_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_black_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------BLACK (SG)---------\n")
        # for entry in black_sg_models[black_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_mixed_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------MIXED (SG)---------\n")
        # for entry in mixed_sg_models[mixed_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_asian_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------ASIAN (SG)---------\n")
        # for entry in asian_sg_models[asian_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
        # word_file.close()
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

for word in interpersonal:
    print(f"\nWORD: {word.upper()}")
    word_file = open(f"results/interpersonal_25/{word}.txt", "a")
    try:
        key = f"{word}_white_cbow"
        word_embeddings[key] = []
        word_file.write("---------WHITE (CBOW)---------\n")
        print(white_cbow_models[white_cbow_model_chosen].wv.similar_by_word(word, 25))
        for entry in white_cbow_models[white_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_black_cbow"
        word_embeddings[key] = []
        word_file.write("---------BLACK (CBOW)---------\n")
        for entry in black_cbow_models[black_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_mixed_cbow"
        word_embeddings[key] = []
        word_file.write("\n---------MIXED (CBOW)---------\n")
        for entry in mixed_cbow_models[mixed_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_asian_cbow"
        word_embeddings[key] = []
        word_file.write("---------ASIAN (CBOW)---------\n")
        for entry in asian_cbow_models[asian_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_white_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------WHITE (SG)---------\n")
        # for entry in black_cbow_models[black_cbow_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_black_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------BLACK (SG)---------\n")
        # for entry in black_sg_models[black_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_mixed_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------MIXED (SG)---------\n")
        # for entry in mixed_sg_models[mixed_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_asian_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------ASIAN (SG)---------\n")
        # for entry in asian_sg_models[asian_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
        # word_file.close()
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

for word in behaviour:
    print(f"\nWORD: {word.upper()}")
    word_file = open(f"results/behaviour_25/{word}.txt", "a")
    try:
        key = f"{word}_white_cbow"
        word_embeddings[key] = []
        word_file.write("---------WHITE (CBOW)---------\n")
        print(white_cbow_models[white_cbow_model_chosen].wv.similar_by_word(word, 25))
        for entry in white_cbow_models[white_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_black_cbow"
        word_embeddings[key] = []
        word_file.write("---------BLACK (CBOW)---------\n")
        for entry in black_cbow_models[black_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_mixed_cbow"
        word_embeddings[key] = []
        word_file.write("\n---------MIXED (CBOW)---------\n")
        for entry in mixed_cbow_models[mixed_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_asian_cbow"
        word_embeddings[key] = []
        word_file.write("---------ASIAN (CBOW)---------\n")
        for entry in asian_cbow_models[asian_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_white_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------WHITE (SG)---------\n")
        # for entry in black_cbow_models[black_cbow_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_black_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------BLACK (SG)---------\n")
        # for entry in black_sg_models[black_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_mixed_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------MIXED (SG)---------\n")
        # for entry in mixed_sg_models[mixed_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_asian_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------ASIAN (SG)---------\n")
        # for entry in asian_sg_models[asian_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
        # word_file.close()
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

for word in clinical:
    print(f"\nWORD: {word.upper()}")
    word_file = open(f"results/clinical_25/{word}.txt", "a")
    try:
        key = f"{word}_white_cbow"
        word_embeddings[key] = []
        word_file.write("---------WHITE (CBOW)---------\n")
        for entry in white_cbow_models[white_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        print("BLACK CBOW: ")
        word_file.write("\n---------BLACK (CBOW)---------\n")
        for entry in black_cbow_models[black_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_mixed_cbow"
        word_embeddings[key] = []
        word_file.write("\n---------MIXED (CBOW)---------\n")
        for entry in mixed_cbow_models[mixed_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_asian_cbow"
        word_embeddings[key] = []
        word_file.write("---------ASIAN (CBOW)---------\n")
        for entry in asian_cbow_models[asian_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_white_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------WHITE (SG)---------\n")
        # for entry in black_cbow_models[black_cbow_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_black_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------BLACK (SG)---------\n")
        # for entry in black_sg_models[black_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_mixed_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------MIXED (SG)---------\n")
        # for entry in mixed_sg_models[mixed_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_asian_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------ASIAN (SG)---------\n")
        # for entry in asian_sg_models[asian_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
        # word_file.close()
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

for word in stereotypes:
    print(f"\nWORD: {word.upper()}")
    word_file = open(f"results/stereotypes_25/{word}.txt", "a")
    try:
        key = f"{word}_white_cbow"
        word_embeddings[key] = []
        word_file.write("---------WHITE (CBOW)---------\n")
        print(white_cbow_models[white_cbow_model_chosen].wv.similar_by_word(word, 25))
        for entry in white_cbow_models[white_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_black_cbow"
        word_embeddings[key] = []
        word_file.write("---------BLACK (CBOW)---------\n")
        for entry in black_cbow_models[black_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_mixed_cbow"
        word_embeddings[key] = []
        word_file.write("\n---------MIXED (CBOW)---------\n")
        for entry in mixed_cbow_models[mixed_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_asian_cbow"
        word_embeddings[key] = []
        word_file.write("---------ASIAN (CBOW)---------\n")
        for entry in asian_cbow_models[asian_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_white_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------WHITE (SG)---------\n")
        # for entry in black_cbow_models[black_cbow_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_black_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------BLACK (SG)---------\n")
        # for entry in black_sg_models[black_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_mixed_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------MIXED (SG)---------\n")
        # for entry in mixed_sg_models[mixed_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_asian_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------ASIAN (SG)---------\n")
        # for entry in asian_sg_models[asian_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
        # word_file.close()
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

for word in subjective:
    print(f"\nWORD: {word.upper()}")
    word_file = open(f"results/subjective_25/{word}.txt", "a")
    try:
        key = f"{word}_white_cbow"
        word_embeddings[key] = []
        word_file.write("---------WHITE (CBOW)---------\n")
        print(white_cbow_models[white_cbow_model_chosen].wv.similar_by_word(word, 25))
        for entry in white_cbow_models[white_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_black_cbow"
        word_embeddings[key] = []
        word_file.write("---------BLACK (CBOW)---------\n")
        for entry in black_cbow_models[black_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_mixed_cbow"
        word_embeddings[key] = []
        word_file.write("\n---------MIXED (CBOW)---------\n")
        for entry in mixed_cbow_models[mixed_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    try:
        key = f"{word}_asian_cbow"
        word_embeddings[key] = []
        word_file.write("---------ASIAN (CBOW)---------\n")
        for entry in asian_cbow_models[asian_cbow_model_chosen].wv.similar_by_word(word, 25):
            try:
                word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                word_embeddings[key].append(entry[0].lower())
            except Exception as error:
                print(f"Could not write to file: {error}")
        word_file.write("\n----------------------------------------------\n")
    except Exception as error:
        print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_white_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------WHITE (SG)---------\n")
        # for entry in black_cbow_models[black_cbow_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_black_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------BLACK (SG)---------\n")
        # for entry in black_sg_models[black_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_mixed_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------MIXED (SG)---------\n")
        # for entry in mixed_sg_models[mixed_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

    # try:
        # key = f"{word}_asian_sg"
        # word_embeddings[key] = []
        # word_file.write("\n---------ASIAN (SG)---------\n")
        # for entry in asian_sg_models[asian_sg_model_chosen].wv.similar_by_word(word, 25):
            # try:
                # word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                # word_embeddings[key].append(entry[0].lower())
            # except Exception as error:
                # print(f"Could not write to file: {error}")
        # word_file.write("\n----------------------------------------------\n")
        # word_file.close()
    # except Exception as error:
        # print(f"no instance of {word} found\n\n")

print(f"LENGTH OF DICTIONARY: {len(word_embeddings.keys())}")
embeddings_file = open(f"results/word_embeddings_15.txt", "w")
embeddings_file.write(json.dumps(word_embeddings))


WORD: DELUSIONAL
[('delusion', 0.9899833798408508), ('Thoughts', 0.9892183542251587), ('unexpected', 0.9886423945426941), ('perceptual', 0.9861484169960022), ('convince', 0.9851548671722412), ('abnormal', 0.9840434789657593), ('image', 0.983711302280426), ('perception', 0.9830425381660461), ('intact', 0.9825755953788757), ('fleeting', 0.9802474975585938), ('subjectively', 0.979609489440918), ('aggressive', 0.9795891046524048), ('pertain', 0.9790951609611511), ('grossly', 0.9776877760887146), ('Childhood', 0.9775781035423279), ('regretful', 0.9771857857704163), ('paranoia', 0.9770001173019409), ('psychomotor', 0.9763065576553345), ('fed', 0.9737668037414551), ('obsession', 0.973608136177063), ('cognition', 0.9735600352287292), ('euthymic', 0.9728654026985168), ('hallucination', 0.9723251461982727), ('denie', 0.9708217978477478), ('anhedonia', 0.9706878066062927)]

WORD: MANIC
[('overdose', 0.9894253611564636), ('anxiety', 0.9888057708740234), ('spell', 0.9865015745162964), ('cocaine', 

75993

In [40]:
# concordances
from nltk.text import Text
from nltk.corpus import gutenberg
nltk.download('gutenberg')
print(white_data.columns)
white_summaries = []
for i in range(white_data.shape[0]):
    toks = (" ".join([t.lower() for t in white_data['tokens'].iloc[i]])).split(" ")
    # print(toks[:10])
    white_summaries.extend(toks)
white_text = Text(white_summaries)
for word in symptoms:
    curr_list = white_text.concordance_list(word)
    word_file = open(f"concordances/white_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in subjective:
    curr_list = white_text.concordance_list(word)
    word_file = open(f"concordances/white_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in stereotypes:
    curr_list = white_text.concordance_list(word)
    word_file = open(f"concordances/white_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in interpersonal:
    curr_list = white_text.concordance_list(word)
    word_file = open(f"concordances/white_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in conditions:
    curr_list = white_text.concordance_list(word)
    word_file = open(f"concordances/white_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in clinical:
    curr_list = white_text.concordance_list(word)
    word_file = open(f"concordances/white_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in behaviour:
    curr_list = white_text.concordance_list(word)
    word_file = open(f"concordances/white_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()

black_summaries = []
for i in range(black_data.shape[0]):
    toks = (" ".join([t.lower() for t in black_data['tokens'].iloc[i]])).split(" ")
    # print(toks[:10])
    black_summaries.extend(toks)
black_text = Text(black_summaries)
for word in symptoms:
    curr_list = black_text.concordance_list(word)
    word_file = open(f"concordances/black_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in subjective:
    curr_list = black_text.concordance_list(word)
    word_file = open(f"concordances/black_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in stereotypes:
    curr_list = black_text.concordance_list(word)
    word_file = open(f"concordances/black_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in interpersonal:
    curr_list = black_text.concordance_list(word)
    word_file = open(f"concordances/black_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in conditions:
    curr_list = black_text.concordance_list(word)
    word_file = open(f"concordances/black_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in clinical:
    curr_list = black_text.concordance_list(word)
    word_file = open(f"concordances/black_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in behaviour:
    curr_list = black_text.concordance_list(word)
    word_file = open(f"concordances/black_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()

asian_summaries = []
for i in range(asian_data.shape[0]):
    toks = (" ".join([t.lower() for t in asian_data['tokens'].iloc[i]])).split(" ")
    # print(toks[:10])
    asian_summaries.extend(toks)
asian_text = Text(asian_summaries)
for word in symptoms:
    curr_list = asian_text.concordance_list(word)
    word_file = open(f"concordances/asian_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in subjective:
    curr_list = asian_text.concordance_list(word)
    word_file = open(f"concordances/asian_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in stereotypes:
    curr_list = asian_text.concordance_list(word)
    word_file = open(f"concordances/asian_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in interpersonal:
    curr_list = asian_text.concordance_list(word)
    word_file = open(f"concordances/asian_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in conditions:
    curr_list = asian_text.concordance_list(word)
    word_file = open(f"concordances/asian_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in clinical:
    curr_list = asian_text.concordance_list(word)
    word_file = open(f"concordances/asian_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in behaviour:
    curr_list = asian_text.concordance_list(word)
    word_file = open(f"concordances/asian_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()

mixed_summaries = []
for i in range(mixed_data.shape[0]):
    toks = (" ".join([t.lower() for t in mixed_data['tokens'].iloc[i]])).split(" ")
    # print(toks[:10])
    mixed_summaries.extend(toks)
mixed_text = Text(mixed_summaries)
for word in symptoms:
    curr_list = mixed_text.concordance_list(word)
    word_file = open(f"concordances/mixed_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in subjective:
    curr_list = mixed_text.concordance_list(word)
    word_file = open(f"concordances/mixed_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in stereotypes:
    curr_list = mixed_text.concordance_list(word)
    word_file = open(f"concordances/mixed_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in interpersonal:
    curr_list = mixed_text.concordance_list(word)
    word_file = open(f"concordances/mixed_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in conditions:
    curr_list = mixed_text.concordance_list(word)
    word_file = open(f"concordances/mixed_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in clinical:
    curr_list = mixed_text.concordance_list(word)
    word_file = open(f"concordances/mixed_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()
for word in behaviour:
    curr_list = mixed_text.concordance_list(word)
    word_file = open(f"concordances/mixed_{word}.txt", "a")
    for entry in curr_list:
        word_file.write(str(entry))
    word_file.close()

[nltk_data] Downloading package gutenberg to /home/jovyan/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


Index(['brcid', 'Gender', 'dob', 'Ethnicity', 'Date_Of_Death', 'Age',
       'Diagnosis_Date', 'Document_Date', 'unique_id', 'Date',
       'Patient_Summary', 'cn_doc_id', 'Summary', 'BRCID',
       'Patient_Summary_sp', 'tokens', 'lemmas', 'lemmas_no_stop'],
      dtype='object')
